In [1]:
import google.generativeai as genai
import os

In [2]:
try:
    # Used to securely store your API key
    from google.colab import userdata

    # Or use `os.getenv('API_KEY')` to fetch an environment variable.
    GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
except ImportError:
    import os
    GOOGLE_API_KEY = os.environ['GOOGLE_API_KEY']

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
        print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-8b-exp-0827


In [3]:
model_info = genai.get_model("models/gemini-1.5-flash")

# Returns the "context window" for the model,
# which is the combined input and output token limits.
print(f"{model_info.input_token_limit=}")
print(f"{model_info.output_token_limit=}")
# ( input_token_limit=30720, output_token_limit=2048 )

model_info.input_token_limit=1048576
model_info.output_token_limit=8192


In [4]:
model = genai.GenerativeModel(model_name="models/gemini-1.5-flash-latest")
model.start_chat()

In [ ]:
def flight_data(departure_id:str,
                arrival_id: str,
                currency: str ,
                outbound_date: str,     #The format is YYYY-MM-DD. e.g. 2024-08-20
                return_date: str,       # The format is YYYY-MM-DD. e.g. 2024-08-20
                flight_type: str,       # flight_type = {"one_way":2,"round_trip":1}
                travel_class: str,      # travel class = {"Economy":1,"premimum Economy":2,"business":3,"first":4}
                adults: int,
                children: int,
                infants: int,
                max_price: int,
                max_flight_duration: int,    # in minutes, like 600 for 10 hours
                max_stops: int    # stops  = {0,Any number of stops (default),1 - Nonstop only,2 - 1 stop or fewer,3 - 2 stops or fewer}
                ):

  """
  This function use Google flight api to fetch the realtime flight data.
  This function requires
  departure city or airport name with expected data,
  arrival city or airport name,
  expected return date or number of days to stay,
  currency type, default is USD,
  travel class e.g: economy, bussines, first,
  flight type e.g : one way, round trip,
  max price in provided currency type,
  max travel duration in minutes,
  max number of stops,
  number of adults,
  number of children,
  number of infants with age,

  This function returns the flight data in json format with details of:
  best flights and other flights(if available)
  flight data contains:
  departure and arrival airport detail like:name, id, time
  duration in minutes,
  airline detail like: name, logo, number
  layovers detail if applicable like : duration, overnight
  price in requested currency
  """
  data = """
There are 4 best flights and 5 other flights available. there details are:

  best flight # 1:

                      Flight Number: EY 201
                      Airline: Etihad
                      Airplane: Airbus A320
                      Class: Economy
                      Legroom: 32 in
                      Duration: 130 minutes
                      Departure: Jinnah International Airport (KHI) at 2024-09-10 20:45
                      Arrival: Zayed International Airport (AUH) at 2024-09-10 21:55
                      Extensions: Above average legroom (32 in), In-seat USB outlet, Stream media to your device, Carbon emissions estimate: 728 kg


                      Flight Number: EY 543
                      Airline: Etihad
                      Airplane: Airbus A321neo
                      Class: Economy
                      Legroom: 30 in
                      Duration: 300 minutes
                      Departure: Zayed International Airport (AUH) at 2024-09-11 02:20
                      Arrival: Istanbul Airport (IST) at 2024-09-11 06:20
                      Extensions: Average legroom (30 in), In-seat USB outlet, Stream media to your device, Carbon emissions estimate: 1094 kg
                      This flight is overnight.


                        Layover at Zayed International Airport (AUH)
                        Duration: 265 minutes
                        This layover is overnight.


        Total Trip Duration: 695 minutes
        Carbon Emissions: 1823000 kg (Typical for this route: 1724000 kg, Difference: 6%)
        Total Price: $2084
        Trip Type: Round trip


  best flight # 2:

                      Flight Number: EY 222
                      Airline: Etihad
                      Airplane: Airbus A320
                      Class: Economy
                      Legroom: 32 in
                      Duration: 130 minutes
                      Departure: Jinnah International Airport (KHI) at 2024-09-10 04:55
                      Arrival: Zayed International Airport (AUH) at 2024-09-10 06:05
                      Extensions: Above average legroom (32 in), In-seat USB outlet, Stream media to your device, Carbon emissions estimate: 728 kg
                      This flight is overnight.


                      Flight Number: EY 541
                      Airline: Etihad
                      Airplane: Boeing 787
                      Class: Economy
                      Legroom: 31 in
                      Duration: 290 minutes
                      Departure: Zayed International Airport (AUH) at 2024-09-10 09:30
                      Arrival: Istanbul Airport (IST) at 2024-09-10 13:20
                      Extensions: Average legroom (31 in), Wi-Fi for a fee, In-seat power & USB outlets, On-demand video, Carbon emissions estimate: 1093 kg


                        Layover at Zayed International Airport (AUH)
                        Duration: 205 minutes


        Total Trip Duration: 625 minutes
        Carbon Emissions: 1821000 kg (Typical for this route: 1724000 kg, Difference: 6%)
        Total Price: $2160
        Trip Type: Round trip


  best flight # 3:

                      Flight Number: FZ 336
                      Airline: flydubai
                      Airplane: Boeing 737
                      Class: Economy
                      Legroom: 30 in
                      Duration: 135 minutes
                      Departure: Jinnah International Airport (KHI) at 2024-09-10 16:40
                      Arrival: Dubai International Airport (DXB) at 2024-09-10 17:55
                      Extensions: Average legroom (30 in), In-seat USB outlet, On-demand video, Carbon emissions estimate: 606 kg


                      Flight Number: FZ 755
                      Airline: flydubai
                      Airplane: Boeing 737MAX 8 Passenger
                      Class: Economy
                      Legroom: 30 in
                      Duration: 295 minutes
                      Departure: Dubai International Airport (DXB) at 2024-09-10 21:15
                      Arrival: Istanbul Airport (IST) at 2024-09-11 01:10
                      Extensions: Average legroom (30 in), In-seat USB outlet, On-demand video, Carbon emissions estimate: 1061 kg


                        Layover at Dubai International Airport (DXB)
                        Duration: 200 minutes


        Total Trip Duration: 630 minutes
        Carbon Emissions: 1668000 kg (Typical for this route: 1724000 kg, Difference: -3%)
        Total Price: $2161
        Trip Type: Round trip


  best flight # 4:

                      Flight Number: EK 601
                      Airline: Emirates
                      Airplane: Boeing 777
                      Class: Economy
                      Legroom: 32 in
                      Duration: 125 minutes
                      Departure: Jinnah International Airport (KHI) at 2024-09-10 12:15
                      Arrival: Dubai International Airport (DXB) at 2024-09-10 13:20
                      Extensions: Above average legroom (32 in), Wi-Fi for a fee, In-seat power & USB outlets, On-demand video, Carbon emissions estimate: 681 kg


                      Flight Number: EK 121
                      Airline: Emirates
                      Airplane: Boeing 777
                      Class: Economy
                      Legroom: 32 in
                      Duration: 275 minutes
                      Departure: Dubai International Airport (DXB) at 2024-09-10 14:20
                      Arrival: Istanbul Airport (IST) at 2024-09-10 17:55
                      Extensions: Above average legroom (32 in), Wi-Fi for a fee, In-seat power & USB outlets, On-demand video, Carbon emissions estimate: 1339 kg


                        Layover at Dubai International Airport (DXB)
                        Duration: 60 minutes


        Total Trip Duration: 460 minutes
        Carbon Emissions: 2021000 kg (Typical for this route: 1724000 kg, Difference: 17%)
        Total Price: $2466
        Trip Type: Round trip


  other flight # 1:

                      Flight Number: EY 201
                      Airline: Etihad
                      Airplane: Airbus A320
                      Class: Economy
                      Legroom: 32 in
                      Duration: 130 minutes
                      Departure: Jinnah International Airport (KHI) at 2024-09-10 20:45
                      Arrival: Zayed International Airport (AUH) at 2024-09-10 21:55
                      Extensions: Above average legroom (32 in), In-seat USB outlet, Stream media to your device, Carbon emissions estimate: 728 kg


                      Flight Number: TK 869
                      Airline: Turkish Airlines
                      Airplane: Airbus A321neo
                      Class: Economy
                      Legroom: 30 in
                      Duration: 295 minutes
                      Departure: Zayed International Airport (AUH) at 2024-09-11 01:30
                      Arrival: Istanbul Airport (IST) at 2024-09-11 05:25
                      Extensions: Average legroom (30 in), Wi-Fi for a fee, In-seat USB outlet, On-demand video, Carbon emissions estimate: 1173 kg
                      This flight is overnight.


                        Layover at Zayed International Airport (AUH)
                        Duration: 215 minutes


        Total Trip Duration: 640 minutes
        Carbon Emissions: 1902000 kg (Typical for this route: 1724000 kg, Difference: 10%)
        Total Price: $2358
        Trip Type: Round trip


  other flight # 2:

                      Flight Number: FZ 332
                      Airline: flydubai
                      Airplane: Boeing 737MAX 8 Passenger
                      Class: Economy
                      Legroom: 30 in
                      Duration: 135 minutes
                      Departure: Jinnah International Airport (KHI) at 2024-09-10 04:55
                      Arrival: Dubai International Airport (DXB) at 2024-09-10 06:10
                      Extensions: Average legroom (30 in), In-seat USB outlet, On-demand video, Carbon emissions estimate: 486 kg
                      This flight is overnight.


                      Flight Number: EK 123
                      Airline: Emirates
                      Airplane: Airbus A380
                      Class: Economy
                      Legroom: 32 in
                      Duration: 280 minutes
                      Departure: Dubai International Airport (DXB) at 2024-09-10 10:45
                      Arrival: Istanbul Airport (IST) at 2024-09-10 14:25
                      Extensions: Above average legroom (32 in), Wi-Fi for a fee, In-seat power & USB outlets, On-demand video, Carbon emissions estimate: 1391 kg


                        Layover at Dubai International Airport (DXB)
                        Duration: 275 minutes


        Total Trip Duration: 690 minutes
        Carbon Emissions: 1879000 kg (Typical for this route: 1724000 kg, Difference: 9%)
        Total Price: $2466
        Trip Type: Round trip


  other flight # 3:

                      Flight Number: QR 611
                      Airline: Qatar Airways
                      Airplane: Airbus A330
                      Class: Economy
                      Legroom: 32 in
                      Duration: 150 minutes
                      Departure: Jinnah International Airport (KHI) at 2024-09-10 10:25
                      Arrival: Hamad International Airport (DOH) at 2024-09-10 10:55
                      Extensions: Above average legroom (32 in), In-seat power & USB outlets, On-demand video, Carbon emissions estimate: 864 kg


                      Flight Number: QR 245
                      Airline: Qatar Airways
                      Airplane: Boeing 777
                      Class: Economy
                      Legroom: 32 in
                      Duration: 255 minutes
                      Departure: Hamad International Airport (DOH) at 2024-09-10 14:20
                      Arrival: Istanbul Airport (IST) at 2024-09-10 18:35
                      Extensions: Above average legroom (32 in), Wi-Fi for a fee, In-seat power & USB outlets, On-demand video, Carbon emissions estimate: 1338 kg


                        Layover at Hamad International Airport (DOH)
                        Duration: 205 minutes


        Total Trip Duration: 610 minutes
        Carbon Emissions: 2202000 kg (Typical for this route: 1724000 kg, Difference: 28%)
        Total Price: $2664
        Trip Type: Round trip


  other flight # 4:

                      Flight Number: GF 751
                      Airline: Gulf Air
                      Airplane: Airbus A321neo
                      Class: Economy
                      Legroom: 31 in
                      Duration: 165 minutes
                      Departure: Jinnah International Airport (KHI) at 2024-09-10 06:50
                      Arrival: Bahrain International Airport (BAH) at 2024-09-10 07:35
                      Extensions: Average legroom (31 in), In-seat power & USB outlets, Stream media to your device, Carbon emissions estimate: 684 kg


                      Flight Number: GF 43
                      Airline: Gulf Air
                      Airplane: Airbus A321neo
                      Class: Economy
                      Legroom: 31 in
                      Duration: 260 minutes
                      Departure: Bahrain International Airport (BAH) at 2024-09-10 09:30
                      Arrival: Istanbul Airport (IST) at 2024-09-10 13:50
                      Extensions: Average legroom (31 in), Free Wi-Fi, In-seat power & USB outlets, On-demand video, Carbon emissions estimate: 1127 kg


                        Layover at Bahrain International Airport (BAH)
                        Duration: 115 minutes


        Total Trip Duration: 540 minutes
        Carbon Emissions: 1812000 kg (Typical for this route: 1724000 kg, Difference: 5%)
        Total Price: $3049
        Trip Type: Round trip


  other flight # 5:

                      Flight Number: TK 709
                      Airline: Turkish Airlines
                      Airplane: Airbus A330
                      Class: Economy
                      Legroom: 33 in
                      Duration: 350 minutes
                      Departure: Jinnah International Airport (KHI) at 2024-09-10 06:00
                      Arrival: Istanbul Airport (IST) at 2024-09-10 09:50
                      Extensions: Above average legroom (33 in), Wi-Fi for a fee, In-seat power & USB outlets, On-demand video, Carbon emissions estimate: 1751 kg


        Total Trip Duration: 350 minutes
        Carbon Emissions: 1751000 kg (Typical for this route: 1724000 kg, Difference: 2%)
        Total Price: $3484
        Trip Type: Round trip
        """

  return data

In [ ]:
available_functions = {
    "flight_data": flight_data,
}

In [ ]:
instruction = """You are trip planner agent and help to make a trip by taking flight data
from function and analyze the function response and make a plan as per user requirment.

"""
# model_name = "gemini-1.5-flash"
model_name = "gemini-1.0-pro"

model_pro = genai.GenerativeModel(
                              model_name=model_name,
                              tools=available_functions.values(),
                              # system_instruction=instruction
)
model_pro

genai.GenerativeModel(
    model_name='models/gemini-1.0-pro',
    generation_config={},
    safety_settings={},
    tools=<google.generativeai.types.content_types.FunctionLibrary object at 0x7a648cba5360>,
    system_instruction=None,
    cached_content=None
)

In [ ]:
instruction = """You are trip planner agent and help to make a trip by taking flight data
from function and analyze the function response and make a plan as per user requirment.
follow the guideline provided in prompt.
"""
model_name = "gemini-1.5-flash"
# model_name = "gemini-1.0-pro"

model_flash = genai.GenerativeModel(
                              model_name=model_name,
                              tools=available_functions.values(),
                              system_instruction=instruction
)
model_flash

genai.GenerativeModel(
    model_name='models/gemini-1.5-flash',
    generation_config={},
    safety_settings={},
    tools=<google.generativeai.types.content_types.FunctionLibrary object at 0x7fa7860bad40>,
    system_instruction='You are trip planner agent and help to make a trip by taking flight data\nfrom function and analyze the function response and make a plan as per user requirment.\nfollow the guideline provided in prompt.\n',
    cached_content=None
)

In [ ]:
prompt = """
I am planning to make a 7 days family trip to istanbul,turkey from karachi,pakistan from 10 of september
my family include me, my wife, a 8 years old son, a daughter of 6 years and a infant of 1.5 years old,
the flight can have max one stop . i prefereed economy class with max budget about $3500.
it will be round trip.
Task:
1- help me to provide a flight plan for my trip to istanbul.
Guide line:
1- user may input city name or airport name, convert it into respective IATA airport code is an uppercase 3-letter code.
e.g for Istanbul airport code the code is "IST" and for Dubai Internationa Airport code is "DXB"
2- if user provide max flight duration, convert it into minutes and if does not proivde max flight duration, set it to 1440 minutes
3- map flight_type as per dictionary  : {2: "one_way",1: "round_trip"}
4- map travel_class as per dictionary : {1: "Economy",2: "premimum Economy",3: "business",4: "first"}
5- map max_stops as per dictionary    : {0: "Any number of stops (default)",1: "Nonstop only",2: "1 stop or fewer",3: "2 stops or fewer}
  """

In [ ]:
response_pro = model_pro.generate_content(prompt)

In [ ]:
response_flash = model_flash.generate_content(prompt)

In [ ]:
response_flash

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "flight_data",
                  "args": {
                    "max_flight_duration": 1440.0,
                    "travel_class": "Economy",
                    "outbound_date": "2023-09-10",
                    "adults": 2.0,
                    "return_date": "2023-09-17",
                    "flight_type": "round_trip",
                    "arrival_id": "IST",
                    "max_stops": 2.0,
                    "currency": "USD",
                    "children": 2.0,
                    "infants": 1.0,
                    "departure_id": "KHI",
                    "max_price": 3500.0
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reaso

In [ ]:
# from datetime import datetime,timedelta
# now = datetime.now()
# outbound_date = now.strftime('%Y-%m-%d')
# print(outbound_date)

# # Convert string to datetime object
# date_object = datetime.strptime(outbound_date, '%Y-%m-%d')

# # Add 7 days
# new_date = date_object + timedelta(days=7)
# new_formatted_date = new_date.strftime('%Y-%m-%d')
# print(new_formatted_date)

2024-08-21
2024-08-28


In [ ]:
from ast import arguments
from datetime import datetime,timedelta
now = datetime.now()
default_outbound_date = now.strftime('%Y-%m-%d')
# Convert string to datetime object
date_object = datetime.strptime(default_outbound_date, '%Y-%m-%d')
# Add 7 days
new_date = date_object + timedelta(days=7)
default_return_date = new_date.strftime('%Y-%m-%d')

for message in response_flash.candidates:
  # print(message.content)
  for part in message.content.parts:
    # print(part)
    try:
      text_content = part.text
      print("Got text")
    except ValueError:
      print("value error text")
    if hasattr(part, 'function_call') and part.function_call is not None:
      print("Got function call")
      try:
        function_call = part.function_call
        if function_call.name in available_functions:
          function_to_call = available_functions[function_call.name]
          print(f"Function {function_call.name} is avalibale")
        else:
          print(f"Function {function_call.name} is not avalibale")

        # Access Arguments
        if hasattr(function_call, "args"):
          print("arguments are present")
          function_arguments = function_call.args
          #Initialized to default values
          departure_id    = "KHI"
          arrival_id      = "DXB"
          currency        = "USD"
          outbound_date   =   default_outbound_date  #The format is YYYY-MM-DD. e.g. 2024-08-20
          return_date     =   default_return_date   # The format is YYYY-MM-DD. e.g. 2024-08-20
          flight_type     =    "1"   # flight_type = {"one_way"2="round_trip"1}
          travel_class    =    1  # travel class = {"Economy"1="premimum Economy"2="business"3="first"4}
          adults          =   1
          children        =   0
          infants         =   0
          max_price       =   "5000"
          max_flight_duration =  1440  # in minutes for e.g 600 for 10 hours
          max_stops       =   0

          # Iterate over each item in 'args'
          for key,value in function_arguments.items():
            if key == "children":
              try:
                children = value
              except Exception as e:
                print(f"Error getting 'children': {e}")
                pass
            if key == "flight_type":
              try:
                flight_type = value
              except Exception as e:
                print(f"Error getting 'flight_type': {e}")
                pass
            if key == "infants":
              try:
                infants = value
              except Exception as e:
                print(f"Error getting 'infants': {e}")
                pass
            if key == "travel_class":
              try:
                travel_class = value
              except Exception as e:
                print(f"Error getting 'travel_class': {e}")
                pass
            if key == "adults":
              try:
                adults = value
              except Exception as e:
                print(f"Error getting 'adults': {e}")
                pass
            if key == "max_flight_duration":
              try:
                max_flight_duration = value
              except Exception as e:
                print(f"Error getting 'max_flight_duration': {e}")
                pass
            if key == "return_date":
              try:
                return_date = value
              except Exception as e:
                print(f"Error getting 'return_date': {e}")
                pass
            if key == "outbound_date":
              try:
                outbound_date = value
              except Exception as e:
                print(f"Error getting 'outbound_date': {e}")
                pass
            if key == "arrival_id":
              try:
                arrival_id = value
              except Exception as e:
                print(f"Error getting 'arrival_id': {e}")
                pass
            if key == "departure_id":
              try:
                departure_id = value
              except Exception as e:
                print(f"Error getting 'departure_id': {e}")
                pass
            if key == "currency":
              try:
                currency = value
              except Exception as e:
                print(f"Error getting 'currency': {e}")
                pass
            if key == "max_price":
              try:
                max_price = value
              except Exception as e:
                print(f"Error getting 'max_price': {e}")
                pass
            if key == "max_stops":
              try:
                max_stops = value
              except Exception as e:
                print(f"Error getting 'max_stops': {e}")
                pass

        #calling google api
        flight_api_response = flight_data(
          departure_id    ,
          arrival_id      ,
          currency        ,
          outbound_date   ,
          return_date     ,
          flight_type     ,
          travel_class    ,
          adults          ,
          children        ,
          infants         ,
          max_price       ,
          max_flight_duration,
          max_stops       , )

        list_content = [prompt, flight_api_response,f"Now continue from last message and helop plan travel in text formt - Funcation Calling completed."]

        second_response = model_flash.generate_content(list_content)

        for message in second_response.candidates:
          print("message.content", message.content)
          for part in message.content.parts:
            print("part", part)
            try:
              text_content = part.text
              print("Got text")
              print("text_content", text_content)
            except ValueError:
              print("value error text")
              pass

      except ValueError:
        print("value error function call")

Got text
Got function call
Function flight_data is avalibale
arguments are present
KHI IST USD 2023-09-10 2023-09-17 round_trip Economy 2.0 2.0 1.0 3500.0 1440.0 2.0
message.content parts {
  text: "Okay, I can help you plan your trip to Istanbul! Based on your requirements, here\'s a flight plan:\n\n**Departure:** Karachi, Pakistan (KHI)\n**Arrival:** Istanbul, Turkey (IST)\n**Travel Dates:** September 10th, 2024, for 7 days\n**Family:** 2 adults, 2 children (8 and 6 years old), 1 infant (1.5 years old)\n**Budget:** $3500\n**Flight Type:** Round trip\n**Travel Class:** Economy\n**Max Stops:** 1\n\n**Best Flight Options:**\n\nLooking at the flight data, I recommend you choose from these best flight options:\n\n* **Flight #1:**  This option has a layover in Abu Dhabi (AUH) but is within your budget, has one stop, and is in economy class.  \n* **Flight #2:** This option also has a layover in Abu Dhabi (AUH) and has one stop, but it might be slightly over budget.\n* **Flight #3:** This op

In [ ]:
from ast import arguments
from datetime import datetime,timedelta
now = datetime.now()
default_outbound_date = now.strftime('%Y-%m-%d')
# Convert string to datetime object
date_object = datetime.strptime(default_outbound_date, '%Y-%m-%d')
# Add 7 days
new_date = date_object + timedelta(days=7)
default_return_date = new_date.strftime('%Y-%m-%d')

for message in response_pro.candidates:
  # print(message.content)
  for part in message.content.parts:
    # print(part)
    try:
      text_content = part.text
      print("Got text")
    except ValueError:
      print("value error text")
    if hasattr(part, 'function_call') and part.function_call is not None:
      print("Got function call")
      try:
        function_call = part.function_call
        if function_call.name in available_functions:
          function_to_call = available_functions[function_call.name]
          print(f"Function {function_call.name} is avalibale")
        else:
          print(f"Function {function_call.name} is not avalibale")

        # Access Arguments
        if hasattr(function_call, "args"):
          print("arguments are present")
          function_arguments = function_call.args
          #Initialized to default values
          departure_id    = "KHI"
          arrival_id      = "DXB"
          currency        = "USD"
          outbound_date   =   default_outbound_date  #The format is YYYY-MM-DD. e.g. 2024-08-20
          return_date     =   default_return_date   # The format is YYYY-MM-DD. e.g. 2024-08-20
          flight_type     =    "1"   # flight_type = {"one_way"2="round_trip"1}
          travel_class    =    1  # travel class = {"Economy"1="premimum Economy"2="business"3="first"4}
          adults          =   1
          children        =   0
          infants         =   0
          max_price       =   "5000"
          max_flight_duration =  1440  # in minutes for e.g 600 for 10 hours
          max_stops       =   0

          # Iterate over each item in 'args'
          for key,value in function_arguments.items():
            if key == "children":
              try:
                children = value
              except Exception as e:
                print(f"Error getting 'children': {e}")
                pass
            if key == "flight_type":
              try:
                flight_type = value
              except Exception as e:
                print(f"Error getting 'flight_type': {e}")
                pass
            if key == "infants":
              try:
                infants = value
              except Exception as e:
                print(f"Error getting 'infants': {e}")
                pass
            if key == "travel_class":
              try:
                travel_class = value
              except Exception as e:
                print(f"Error getting 'travel_class': {e}")
                pass
            if key == "adults":
              try:
                adults = value
              except Exception as e:
                print(f"Error getting 'adults': {e}")
                pass
            if key == "max_flight_duration":
              try:
                max_flight_duration = value
              except Exception as e:
                print(f"Error getting 'max_flight_duration': {e}")
                pass
            if key == "return_date":
              try:
                return_date = value
              except Exception as e:
                print(f"Error getting 'return_date': {e}")
                pass
            if key == "outbound_date":
              try:
                outbound_date = value
              except Exception as e:
                print(f"Error getting 'outbound_date': {e}")
                pass
            if key == "arrival_id":
              try:
                arrival_id = value
              except Exception as e:
                print(f"Error getting 'arrival_id': {e}")
                pass
            if key == "departure_id":
              try:
                departure_id = value
              except Exception as e:
                print(f"Error getting 'departure_id': {e}")
                pass
            if key == "currency":
              try:
                currency = value
              except Exception as e:
                print(f"Error getting 'currency': {e}")
                pass
            if key == "max_price":
              try:
                max_price = value
              except Exception as e:
                print(f"Error getting 'max_price': {e}")
                pass
            if key == "max_stops":
              try:
                max_stops = value
              except Exception as e:
                print(f"Error getting 'max_stops': {e}")
                pass
        #calling google api
        flight_api_response = flight_data(
          departure_id    ,
          arrival_id      ,
          currency        ,
          outbound_date   ,
          return_date     ,
          flight_type     ,
          travel_class    ,
          adults          ,
          children        ,
          infants         ,
          max_price       ,
          max_flight_duration,
          max_stops       , )

        list_content = [prompt, flight_api_response,f"Now continue from last message and helop plan travel in text formt - Funcation Calling completed."]

        second_response = model_flash.generate_content(list_content)

        for message in second_response.candidates:
          # print("message.content", message.content)
          for part in message.content.parts:
            # print("part", part)
            try:
              text_content = part.text
              # print("Got text")
              print("****text_content:****", text_content)
            except ValueError:
              print("value error text")
              pass

      except ValueError:
        print("value error function call")

Got text
Got function call
Function flight_data is avalibale
arguments are present
KHI IST USD 2023-09-10 2023-09-17 1.0 1.0 2.0 2.0 1.0 3500.0 1440.0 2.0
message.content parts {
  text: "I can help you plan your trip to Istanbul! \n\nBased on your requirements, here\'s a flight plan:\n\n**Flight Details:**\n\n* **Departure:** Karachi, Pakistan (KHI)\n* **Arrival:** Istanbul, Turkey (IST)\n* **Departure Date:** September 10, 2024\n* **Return Date:** September 17, 2024 (7 days trip)\n* **Travel Class:** Economy\n* **Max Budget:** $3500 \n* **Max Stops:** 1 stop \n* **Number of Adults:** 2 \n* **Number of Children:** 2 (8 years old and 6 years old)\n* **Number of Infants:** 1 (1.5 years old)\n\n**Best Option:** \n\nFlight #1 (Total Price: $2084)\n\n* **Outbound Flight:**\n    * EY 201 (Etihad): KHI to AUH (130 minutes)\n    * EY 543 (Etihad): AUH to IST (300 minutes, overnight)\n    * Layover at AUH (265 minutes, overnight) \n* **Return Flight:**\n    * EY 543 (Etihad): IST to AUH (300 m

In [10]:
import IPython
from IPython.display import Markdown

In [ ]:
response.candidates[0].content.parts

[function_call {
  name: "flight_data"
  args {
    fields {
      key: "adults"
      value {
        number_value: 2.0
      }
    }
    fields {
      key: "arrival_id"
      value {
        string_value: "IST"
      }
    }
    fields {
      key: "children"
      value {
        number_value: 2.0
      }
    }
    fields {
      key: "currency"
      value {
        string_value: "USD"
      }
    }
    fields {
      key: "departure_id"
      value {
        string_value: "KHI"
      }
    }
    fields {
      key: "flight_type"
      value {
        string_value: "round trip"
      }
    }
    fields {
      key: "infants"
      value {
        number_value: 1.0
      }
    }
    fields {
      key: "max_flight_duration"
      value {
        number_value: 1000.0
      }
    }
    fields {
      key: "max_price"
      value {
        number_value: 3500.0
      }
    }
    fields {
      key: "max_stops"
      value {
        number_value: 1.0
      }
    }
    fields {
      key

In [ ]:
e="e"
args = response.candidates[0].content.parts[0].function_call.args
for key, value in args.items():
    #  print("Key:", key, "Value:", value)
     print(f"""if key == "{key}": \n\ttry:\n\t\t {key} = value\n\texcept Exception as {e}:\n\t\tprint(f"Error getting '{key}': {{e}}")\n\t\tpass""")

if key == "children": 
	try:
		 children = value
	except Exception as e:
		print(f"Error getting 'children': {e}")
		pass
if key == "flight_type": 
	try:
		 flight_type = value
	except Exception as e:
		print(f"Error getting 'flight_type': {e}")
		pass
if key == "infants": 
	try:
		 infants = value
	except Exception as e:
		print(f"Error getting 'infants': {e}")
		pass
if key == "travel_class": 
	try:
		 travel_class = value
	except Exception as e:
		print(f"Error getting 'travel_class': {e}")
		pass
if key == "adults": 
	try:
		 adults = value
	except Exception as e:
		print(f"Error getting 'adults': {e}")
		pass
if key == "max_flight_duration": 
	try:
		 max_flight_duration = value
	except Exception as e:
		print(f"Error getting 'max_flight_duration': {e}")
		pass
if key == "return_date": 
	try:
		 return_date = value
	except Exception as e:
		print(f"Error getting 'return_date': {e}")
		pass
if key == "outbound_date": 
	try:
		 outbound_date = value
	except Exception as e:
		print(f

In [ ]:
response_flash.candidates[0].content.parts

[function_call {
  name: "flight_data"
  args {
    fields {
      key: "adults"
      value {
        number_value: 2.0
      }
    }
    fields {
      key: "arrival_id"
      value {
        string_value: "IST"
      }
    }
    fields {
      key: "children"
      value {
        number_value: 2.0
      }
    }
    fields {
      key: "currency"
      value {
        string_value: "USD"
      }
    }
    fields {
      key: "departure_id"
      value {
        string_value: "KHI"
      }
    }
    fields {
      key: "flight_type"
      value {
        string_value: "round_trip"
      }
    }
    fields {
      key: "infants"
      value {
        number_value: 1.0
      }
    }
    fields {
      key: "max_flight_duration"
      value {
        number_value: 1440.0
      }
    }
    fields {
      key: "max_price"
      value {
        number_value: 3500.0
      }
    }
    fields {
      key: "max_stops"
      value {
        number_value: 2.0
      }
    }
    fields {
      key

serpapi-input

In [ ]:
api_response ={
   "search_metadata":{
      "id":"66c57adda2510416352cc424",
      "status":"Success",
      "json_endpoint":"https://serpapi.com/searches/27f0f34d1f8eaa77/66c57adda2510416352cc424.json",
      "created_at":"2024-08-21 05:27:57 UTC",
      "processed_at":"2024-08-21 05:27:57 UTC",
      "google_flights_url":"https://www.google.com/travel/flights?hl=en&gl=pk&curr=USD&tfs=CBwQAhojEgoyMDI0LTA5LTEwKAJgoAZqBwgBEgNLSElyBwgBEgNJU1QaIxIKMjAyNC0wOS0zMCgCYKAGagcIARIDSVNUcgcIARIDS0hJQgYBAQICAwRIAWCIJ3ABmAEB",
      "raw_html_file":"https://serpapi.com/searches/27f0f34d1f8eaa77/66c57adda2510416352cc424.html",
      "prettify_html_file":"https://serpapi.com/searches/27f0f34d1f8eaa77/66c57adda2510416352cc424.prettify",
      "total_time_taken":3.89
   },
   "search_parameters":{
      "engine":"google_flights",
      "hl":"en",
      "gl":"pk",
      "type":"1",
      "departure_id":"KHI",
      "arrival_id":"IST",
      "outbound_date":"2024-09-10",
      "return_date":"2024-09-30",
      "travel_class":1,
      "adults":2,
      "children":2,
      "infants_in_seat":1,
      "infants_on_lap":1,
      "stops":2,
      "max_price":"5000",
      "max_duration":800,
      "currency":"USD"
   },
   "best_flights":[
      {
         "flights":[
            {
               "departure_airport":{
                  "name":"Jinnah International Airport",
                  "id":"KHI",
                  "time":"2024-09-10 20:45"
               },
               "arrival_airport":{
                  "name":"Zayed International Airport",
                  "id":"AUH",
                  "time":"2024-09-10 21:55"
               },
               "duration":130,
               "airplane":"Airbus A320",
               "airline":"Etihad",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/EY.png",
               "travel_class":"Economy",
               "flight_number":"EY 201",
               "legroom":"32 in",
               "extensions":[
                  "Above average legroom (32 in)",
                  "In-seat USB outlet",
                  "Stream media to your device",
                  "Carbon emissions estimate: 728 kg"
               ]
            },
            {
               "departure_airport":{
                  "name":"Zayed International Airport",
                  "id":"AUH",
                  "time":"2024-09-11 02:20"
               },
               "arrival_airport":{
                  "name":"Istanbul Airport",
                  "id":"IST",
                  "time":"2024-09-11 06:20"
               },
               "duration":300,
               "airplane":"Airbus A321neo",
               "airline":"Etihad",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/EY.png",
               "travel_class":"Economy",
               "flight_number":"EY 543",
               "legroom":"30 in",
               "extensions":[
                  "Average legroom (30 in)",
                  "In-seat USB outlet",
                  "Stream media to your device",
                  "Carbon emissions estimate: 1094 kg"
               ],
               "overnight":True
            }
         ],
         "layovers":[
            {
               "duration":265,
               "name":"Zayed International Airport",
               "id":"AUH",
               "overnight":True
            }
         ],
         "total_duration":695,
         "carbon_emissions":{
            "this_flight":1823000,
            "typical_for_this_route":1724000,
            "difference_percent":6
         },
         "price":2084,
         "type":"Round trip",
         "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/EY.png",
         "departure_token":"WyJDalJJUW1wU1QzZEtWSE0wUlRoQlFqTnJNbWRDUnkwdExTMHRMUzB0TFhsc1pXNHlNa0ZCUVVGQlIySkdaWFE0UmpSU2VWbEJFZ3RGV1RJd01YeEZXVFUwTXhvTENQUGJEQkFDR2dOVlUwUTRISER6Mnd3PSIsW1siS0hJIiwiMjAyNC0wOS0xMCIsIkFVSCIsbnVsbCwiRVkiLCIyMDEiXSxbIkFVSCIsIjIwMjQtMDktMTEiLCJJU1QiLG51bGwsIkVZIiwiNTQzIl1dXQ=="
      },
      {
         "flights":[
            {
               "departure_airport":{
                  "name":"Jinnah International Airport",
                  "id":"KHI",
                  "time":"2024-09-10 04:55"
               },
               "arrival_airport":{
                  "name":"Zayed International Airport",
                  "id":"AUH",
                  "time":"2024-09-10 06:05"
               },
               "duration":130,
               "airplane":"Airbus A320",
               "airline":"Etihad",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/EY.png",
               "travel_class":"Economy",
               "flight_number":"EY 222",
               "legroom":"32 in",
               "extensions":[
                  "Above average legroom (32 in)",
                  "In-seat USB outlet",
                  "Stream media to your device",
                  "Carbon emissions estimate: 728 kg"
               ],
               "overnight":True
            },
            {
               "departure_airport":{
                  "name":"Zayed International Airport",
                  "id":"AUH",
                  "time":"2024-09-10 09:30"
               },
               "arrival_airport":{
                  "name":"Istanbul Airport",
                  "id":"IST",
                  "time":"2024-09-10 13:20"
               },
               "duration":290,
               "airplane":"Boeing 787",
               "airline":"Etihad",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/EY.png",
               "travel_class":"Economy",
               "flight_number":"EY 541",
               "legroom":"31 in",
               "extensions":[
                  "Average legroom (31 in)",
                  "Wi-Fi for a fee",
                  "In-seat power & USB outlets",
                  "On-demand video",
                  "Carbon emissions estimate: 1093 kg"
               ]
            }
         ],
         "layovers":[
            {
               "duration":205,
               "name":"Zayed International Airport",
               "id":"AUH"
            }
         ],
         "total_duration":625,
         "carbon_emissions":{
            "this_flight":1821000,
            "typical_for_this_route":1724000,
            "difference_percent":6
         },
         "price":2160,
         "type":"Round trip",
         "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/EY.png",
         "departure_token":"WyJDalJJUW1wU1QzZEtWSE0wUlRoQlFqTnJNbWRDUnkwdExTMHRMUzB0TFhsc1pXNHlNa0ZCUVVGQlIySkdaWFE0UmpSU2VWbEJFZ3RGV1RJeU1ueEZXVFUwTVJvTENPK1dEUkFDR2dOVlUwUTRISER2bGcwPSIsW1siS0hJIiwiMjAyNC0wOS0xMCIsIkFVSCIsbnVsbCwiRVkiLCIyMjIiXSxbIkFVSCIsIjIwMjQtMDktMTAiLCJJU1QiLG51bGwsIkVZIiwiNTQxIl1dXQ=="
      },
      {
         "flights":[
            {
               "departure_airport":{
                  "name":"Jinnah International Airport",
                  "id":"KHI",
                  "time":"2024-09-10 16:40"
               },
               "arrival_airport":{
                  "name":"Dubai International Airport",
                  "id":"DXB",
                  "time":"2024-09-10 17:55"
               },
               "duration":135,
               "airplane":"Boeing 737",
               "airline":"flydubai",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/FZ.png",
               "travel_class":"Economy",
               "flight_number":"FZ 336",
               "ticket_also_sold_by":[
                  "Emirates"
               ],
               "legroom":"30 in",
               "extensions":[
                  "Average legroom (30 in)",
                  "In-seat USB outlet",
                  "On-demand video",
                  "Carbon emissions estimate: 606 kg"
               ],
               "often_delayed_by_over_30_min":True
            },
            {
               "departure_airport":{
                  "name":"Dubai International Airport",
                  "id":"DXB",
                  "time":"2024-09-10 21:15"
               },
               "arrival_airport":{
                  "name":"Istanbul Airport",
                  "id":"IST",
                  "time":"2024-09-11 01:10"
               },
               "duration":295,
               "airplane":"Boeing 737MAX 8 Passenger",
               "airline":"flydubai",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/FZ.png",
               "travel_class":"Economy",
               "flight_number":"FZ 755",
               "ticket_also_sold_by":[
                  "Emirates"
               ],
               "legroom":"30 in",
               "extensions":[
                  "Average legroom (30 in)",
                  "In-seat USB outlet",
                  "On-demand video",
                  "Carbon emissions estimate: 1061 kg"
               ]
            }
         ],
         "layovers":[
            {
               "duration":200,
               "name":"Dubai International Airport",
               "id":"DXB"
            }
         ],
         "total_duration":630,
         "carbon_emissions":{
            "this_flight":1668000,
            "typical_for_this_route":1724000,
            "difference_percent":-3
         },
         "price":2161,
         "type":"Round trip",
         "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/FZ.png",
         "departure_token":"WyJDalJJUW1wU1QzZEtWSE0wUlRoQlFqTnJNbWRDUnkwdExTMHRMUzB0TFhsc1pXNHlNa0ZCUVVGQlIySkdaWFE0UmpSU2VWbEJFZ3RHV2pNek5ueEdXamMxTlJvTENQMlhEUkFDR2dOVlUwUTRISEQ5bHcwPSIsW1siS0hJIiwiMjAyNC0wOS0xMCIsIkRYQiIsbnVsbCwiRloiLCIzMzYiXSxbIkRYQiIsIjIwMjQtMDktMTAiLCJJU1QiLG51bGwsIkZaIiwiNzU1Il1dXQ=="
      },
      {
         "flights":[
            {
               "departure_airport":{
                  "name":"Jinnah International Airport",
                  "id":"KHI",
                  "time":"2024-09-10 12:15"
               },
               "arrival_airport":{
                  "name":"Dubai International Airport",
                  "id":"DXB",
                  "time":"2024-09-10 13:20"
               },
               "duration":125,
               "airplane":"Boeing 777",
               "airline":"Emirates",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/EK.png",
               "travel_class":"Economy",
               "flight_number":"EK 601",
               "legroom":"32 in",
               "extensions":[
                  "Above average legroom (32 in)",
                  "Wi-Fi for a fee",
                  "In-seat power & USB outlets",
                  "On-demand video",
                  "Carbon emissions estimate: 681 kg"
               ]
            },
            {
               "departure_airport":{
                  "name":"Dubai International Airport",
                  "id":"DXB",
                  "time":"2024-09-10 14:20"
               },
               "arrival_airport":{
                  "name":"Istanbul Airport",
                  "id":"IST",
                  "time":"2024-09-10 17:55"
               },
               "duration":275,
               "airplane":"Boeing 777",
               "airline":"Emirates",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/EK.png",
               "travel_class":"Economy",
               "flight_number":"EK 121",
               "legroom":"32 in",
               "extensions":[
                  "Above average legroom (32 in)",
                  "Wi-Fi for a fee",
                  "In-seat power & USB outlets",
                  "On-demand video",
                  "Carbon emissions estimate: 1339 kg"
               ]
            }
         ],
         "layovers":[
            {
               "duration":60,
               "name":"Dubai International Airport",
               "id":"DXB"
            }
         ],
         "total_duration":460,
         "carbon_emissions":{
            "this_flight":2021000,
            "typical_for_this_route":1724000,
            "difference_percent":17
         },
         "price":2466,
         "type":"Round trip",
         "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/EK.png",
         "departure_token":"WyJDalJJUW1wU1QzZEtWSE0wUlRoQlFqTnJNbWRDUnkwdExTMHRMUzB0TFhsc1pXNHlNa0ZCUVVGQlIySkdaWFE0UmpSU2VWbEJFZ3RGU3pZd01YeEZTekV5TVJvTENQNkZEeEFDR2dOVlUwUTRISEQraFE4PSIsW1siS0hJIiwiMjAyNC0wOS0xMCIsIkRYQiIsbnVsbCwiRUsiLCI2MDEiXSxbIkRYQiIsIjIwMjQtMDktMTAiLCJJU1QiLG51bGwsIkVLIiwiMTIxIl1dXQ=="
      }
   ],
   "other_flights":[
      {
         "flights":[
            {
               "departure_airport":{
                  "name":"Jinnah International Airport",
                  "id":"KHI",
                  "time":"2024-09-10 20:45"
               },
               "arrival_airport":{
                  "name":"Zayed International Airport",
                  "id":"AUH",
                  "time":"2024-09-10 21:55"
               },
               "duration":130,
               "airplane":"Airbus A320",
               "airline":"Etihad",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/EY.png",
               "travel_class":"Economy",
               "flight_number":"EY 201",
               "legroom":"32 in",
               "extensions":[
                  "Above average legroom (32 in)",
                  "In-seat USB outlet",
                  "Stream media to your device",
                  "Carbon emissions estimate: 728 kg"
               ]
            },
            {
               "departure_airport":{
                  "name":"Zayed International Airport",
                  "id":"AUH",
                  "time":"2024-09-11 01:30"
               },
               "arrival_airport":{
                  "name":"Istanbul Airport",
                  "id":"IST",
                  "time":"2024-09-11 05:25"
               },
               "duration":295,
               "airplane":"Airbus A321neo",
               "airline":"Turkish Airlines",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/TK.png",
               "travel_class":"Economy",
               "flight_number":"TK 869",
               "legroom":"30 in",
               "extensions":[
                  "Average legroom (30 in)",
                  "Wi-Fi for a fee",
                  "In-seat USB outlet",
                  "On-demand video",
                  "Carbon emissions estimate: 1173 kg"
               ],
               "overnight":True
            }
         ],
         "layovers":[
            {
               "duration":215,
               "name":"Zayed International Airport",
               "id":"AUH"
            }
         ],
         "total_duration":640,
         "carbon_emissions":{
            "this_flight":1902000,
            "typical_for_this_route":1724000,
            "difference_percent":10
         },
         "price":2358,
         "type":"Round trip",
         "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/multi.png",
         "departure_token":"WyJDalJJUW1wU1QzZEtWSE0wUlRoQlFqTnJNbWRDUnkwdExTMHRMUzB0TFhsc1pXNHlNa0ZCUVVGQlIySkdaWFE0UmpSU2VWbEJFZ3RGV1RJd01YeFVTemcyT1JvTENQU3hEaEFDR2dOVlUwUTRISEQwc1E0PSIsW1siS0hJIiwiMjAyNC0wOS0xMCIsIkFVSCIsbnVsbCwiRVkiLCIyMDEiXSxbIkFVSCIsIjIwMjQtMDktMTEiLCJJU1QiLG51bGwsIlRLIiwiODY5Il1dXQ=="
      },
      {
         "flights":[
            {
               "departure_airport":{
                  "name":"Jinnah International Airport",
                  "id":"KHI",
                  "time":"2024-09-10 04:55"
               },
               "arrival_airport":{
                  "name":"Dubai International Airport",
                  "id":"DXB",
                  "time":"2024-09-10 06:10"
               },
               "duration":135,
               "airplane":"Boeing 737MAX 8 Passenger",
               "airline":"flydubai",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/FZ.png",
               "travel_class":"Economy",
               "flight_number":"FZ 332",
               "ticket_also_sold_by":[
                  "Emirates"
               ],
               "legroom":"30 in",
               "extensions":[
                  "Average legroom (30 in)",
                  "In-seat USB outlet",
                  "On-demand video",
                  "Carbon emissions estimate: 486 kg"
               ],
               "overnight":True
            },
            {
               "departure_airport":{
                  "name":"Dubai International Airport",
                  "id":"DXB",
                  "time":"2024-09-10 10:45"
               },
               "arrival_airport":{
                  "name":"Istanbul Airport",
                  "id":"IST",
                  "time":"2024-09-10 14:25"
               },
               "duration":280,
               "airplane":"Airbus A380",
               "airline":"Emirates",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/EK.png",
               "travel_class":"Economy",
               "flight_number":"EK 123",
               "legroom":"32 in",
               "extensions":[
                  "Above average legroom (32 in)",
                  "Wi-Fi for a fee",
                  "In-seat power & USB outlets",
                  "On-demand video",
                  "Carbon emissions estimate: 1391 kg"
               ],
               "often_delayed_by_over_30_min":True
            }
         ],
         "layovers":[
            {
               "duration":275,
               "name":"Dubai International Airport",
               "id":"DXB"
            }
         ],
         "total_duration":690,
         "carbon_emissions":{
            "this_flight":1879000,
            "typical_for_this_route":1724000,
            "difference_percent":9
         },
         "price":2466,
         "type":"Round trip",
         "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/multi.png",
         "departure_token":"WyJDalJJUW1wU1QzZEtWSE0wUlRoQlFqTnJNbWRDUnkwdExTMHRMUzB0TFhsc1pXNHlNa0ZCUVVGQlIySkdaWFE0UmpSU2VWbEJFZ3RHV2pNek1ueEZTekV5TXhvTENQNkZEeEFDR2dOVlUwUTRISEQraFE4PSIsW1siS0hJIiwiMjAyNC0wOS0xMCIsIkRYQiIsbnVsbCwiRloiLCIzMzIiXSxbIkRYQiIsIjIwMjQtMDktMTAiLCJJU1QiLG51bGwsIkVLIiwiMTIzIl1dXQ=="
      },
      {
         "flights":[
            {
               "departure_airport":{
                  "name":"Jinnah International Airport",
                  "id":"KHI",
                  "time":"2024-09-10 10:25"
               },
               "arrival_airport":{
                  "name":"Hamad International Airport",
                  "id":"DOH",
                  "time":"2024-09-10 10:55"
               },
               "duration":150,
               "airplane":"Airbus A330",
               "airline":"Qatar Airways",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/QR.png",
               "travel_class":"Economy",
               "flight_number":"QR 611",
               "legroom":"32 in",
               "extensions":[
                  "Above average legroom (32 in)",
                  "In-seat power & USB outlets",
                  "On-demand video",
                  "Carbon emissions estimate: 864 kg"
               ]
            },
            {
               "departure_airport":{
                  "name":"Hamad International Airport",
                  "id":"DOH",
                  "time":"2024-09-10 14:20"
               },
               "arrival_airport":{
                  "name":"Istanbul Airport",
                  "id":"IST",
                  "time":"2024-09-10 18:35"
               },
               "duration":255,
               "airplane":"Boeing 777",
               "airline":"Qatar Airways",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/QR.png",
               "travel_class":"Economy",
               "flight_number":"QR 245",
               "legroom":"32 in",
               "extensions":[
                  "Above average legroom (32 in)",
                  "Wi-Fi for a fee",
                  "In-seat power & USB outlets",
                  "On-demand video",
                  "Carbon emissions estimate: 1338 kg"
               ]
            }
         ],
         "layovers":[
            {
               "duration":205,
               "name":"Hamad International Airport",
               "id":"DOH"
            }
         ],
         "total_duration":610,
         "carbon_emissions":{
            "this_flight":2202000,
            "typical_for_this_route":1724000,
            "difference_percent":28
         },
         "price":2664,
         "type":"Round trip",
         "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/QR.png",
         "departure_token":"WyJDalJJUW1wU1QzZEtWSE0wUlRoQlFqTnJNbWRDUnkwdExTMHRMUzB0TFhsc1pXNHlNa0ZCUVVGQlIySkdaWFE0UmpSU2VWbEJFZ3RSVWpZeE1YeFJVakkwTlJvTENJMmhFQkFDR2dOVlUwUTRISENOb1JBPSIsW1siS0hJIiwiMjAyNC0wOS0xMCIsIkRPSCIsbnVsbCwiUVIiLCI2MTEiXSxbIkRPSCIsIjIwMjQtMDktMTAiLCJJU1QiLG51bGwsIlFSIiwiMjQ1Il1dXQ=="
      },
      {
         "flights":[
            {
               "departure_airport":{
                  "name":"Jinnah International Airport",
                  "id":"KHI",
                  "time":"2024-09-10 06:50"
               },
               "arrival_airport":{
                  "name":"Bahrain International Airport",
                  "id":"BAH",
                  "time":"2024-09-10 07:35"
               },
               "duration":165,
               "airplane":"Airbus A321neo",
               "airline":"Gulf Air",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/GF.png",
               "travel_class":"Economy",
               "flight_number":"GF 751",
               "legroom":"31 in",
               "extensions":[
                  "Average legroom (31 in)",
                  "In-seat power & USB outlets",
                  "Stream media to your device",
                  "Carbon emissions estimate: 684 kg"
               ]
            },
            {
               "departure_airport":{
                  "name":"Bahrain International Airport",
                  "id":"BAH",
                  "time":"2024-09-10 09:30"
               },
               "arrival_airport":{
                  "name":"Istanbul Airport",
                  "id":"IST",
                  "time":"2024-09-10 13:50"
               },
               "duration":260,
               "airplane":"Airbus A321neo",
               "airline":"Gulf Air",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/GF.png",
               "travel_class":"Economy",
               "flight_number":"GF 43",
               "legroom":"31 in",
               "extensions":[
                  "Average legroom (31 in)",
                  "Free Wi-Fi",
                  "In-seat power & USB outlets",
                  "On-demand video",
                  "Carbon emissions estimate: 1127 kg"
               ]
            }
         ],
         "layovers":[
            {
               "duration":115,
               "name":"Bahrain International Airport",
               "id":"BAH"
            }
         ],
         "total_duration":540,
         "carbon_emissions":{
            "this_flight":1812000,
            "typical_for_this_route":1724000,
            "difference_percent":5
         },
         "price":3049,
         "type":"Round trip",
         "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/GF.png",
         "departure_token":"WyJDalJJUW1wU1QzZEtWSE0wUlRoQlFqTnJNbWRDUnkwdExTMHRMUzB0TFhsc1pXNHlNa0ZCUVVGQlIySkdaWFE0UmpSU2VWbEJFZ3hIUmpjMU1YeEhSalF6SXpFYUN3akt6UklRQWhvRFZWTkVPQnh3eXMwUyIsW1siS0hJIiwiMjAyNC0wOS0xMCIsIkJBSCIsbnVsbCwiR0YiLCI3NTEiXSxbIkJBSCIsIjIwMjQtMDktMTAiLCJJU1QiLG51bGwsIkdGIiwiNDMiXV1d"
      },
      {
         "flights":[
            {
               "departure_airport":{
                  "name":"Jinnah International Airport",
                  "id":"KHI",
                  "time":"2024-09-10 06:00"
               },
               "arrival_airport":{
                  "name":"Istanbul Airport",
                  "id":"IST",
                  "time":"2024-09-10 09:50"
               },
               "duration":350,
               "airplane":"Airbus A330",
               "airline":"Turkish Airlines",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/TK.png",
               "travel_class":"Economy",
               "flight_number":"TK 709",
               "legroom":"33 in",
               "extensions":[
                  "Above average legroom (33 in)",
                  "Wi-Fi for a fee",
                  "In-seat power & USB outlets",
                  "On-demand video",
                  "Carbon emissions estimate: 1751 kg"
               ]
            }
         ],
         "total_duration":350,
         "carbon_emissions":{
            "this_flight":1751000,
            "typical_for_this_route":1724000,
            "difference_percent":2
         },
         "price":3484,
         "type":"Round trip",
         "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/TK.png",
         "departure_token":"WyJDalJJUW1wU1QzZEtWSE0wUlRoQlFqTnJNbWRDUnkwdExTMHRMUzB0TFhsc1pXNHlNa0ZCUVVGQlIySkdaWFE0UmpSU2VWbEJFZ2RVU3pjd09TTXlHZ3NJMEtFVkVBSWFBMVZUUkRnY2NOQ2hGUT09IixbWyJLSEkiLCIyMDI0LTA5LTEwIiwiSVNUIixudWxsLCJUSyIsIjcwOSJdXV0="
      }
   ]
}

In [ ]:
best_flights= api_response["best_flights"]
other_flights= api_response["other_flights"]

In [ ]:
other_flights[0]

{'flights': [{'departure_airport': {'name': 'Jinnah International Airport',
    'id': 'KHI',
    'time': '2024-09-10 20:45'},
   'arrival_airport': {'name': 'Zayed International Airport',
    'id': 'AUH',
    'time': '2024-09-10 21:55'},
   'duration': 130,
   'airplane': 'Airbus A320',
   'airline': 'Etihad',
   'airline_logo': 'https://www.gstatic.com/flights/airline_logos/70px/EY.png',
   'travel_class': 'Economy',
   'flight_number': 'EY 201',
   'legroom': '32 in',
   'extensions': ['Above average legroom (32 in)',
    'In-seat USB outlet',
    'Stream media to your device',
    'Carbon emissions estimate: 728 kg']},
  {'departure_airport': {'name': 'Zayed International Airport',
    'id': 'AUH',
    'time': '2024-09-11 01:30'},
   'arrival_airport': {'name': 'Istanbul Airport',
    'id': 'IST',
    'time': '2024-09-11 05:25'},
   'duration': 295,
   'airplane': 'Airbus A321neo',
   'airline': 'Turkish Airlines',
   'airline_logo': 'https://www.gstatic.com/flights/airline_logos/70

In [5]:
# Function to parse and format the flight data
def parse_flight_data(flight_data):
  flights = flight_data['flights']
  if 'layovers' in flights:
    layovers = flight_data['layovers']
  else:
    layovers = ['no layovers, it is a direct flight']
  total_duration = flight_data['total_duration']
  carbon_emissions = flight_data['carbon_emissions']
  price = flight_data['price']
  trip_type = flight_data['type']

  plain_text_output = []

  for flight in flights:
        departure = flight['departure_airport']
        arrival = flight['arrival_airport']

        flight_info = f"""
                      Flight Number: {flight['flight_number']}
                      Airline: {flight['airline']}
                      Airplane: {flight['airplane']}
                      Class: {flight['travel_class']}
                      Legroom: {flight['legroom']}
                      Duration: {flight['duration']} minutes
                      Departure: {departure['name']} ({departure['id']}) at {departure['time']}
                      Arrival: {arrival['name']} ({arrival['id']}) at {arrival['time']}
                      Extensions: {', '.join(flight['extensions'])}
                      """
        if 'overnight' in flight and flight['overnight']:
            flight_info += "This flight is overnight.\n"

        plain_text_output.append(flight_info)


  if 'layovers' in flights:
    for layover in layovers:
          layover_info = f"""
                        Layover at {layover['name']} ({layover['id']})
                        Duration: {layover['duration']} minutes
                        """
          if 'overnight' in layover and layover['overnight']:
              layover_info += "This layover is overnight.\n"

          plain_text_output.append(layover_info)

  summary = f"""
        Total Trip Duration: {total_duration} minutes
        Carbon Emissions: {carbon_emissions['this_flight']} kg (Typical for this route: {carbon_emissions['typical_for_this_route']} kg, Difference: {carbon_emissions['difference_percent']}%)
        Total Price: ${price}
        Trip Type: {trip_type}
        """
  plain_text_output.append(summary)

  return "\n".join(plain_text_output)



In [ ]:
# Convert the JSON data to plain text
def parse_flight_data_formater(best_flights,other_flights):
  flight_data_parsed= f"""
  There are {len(best_flights)} best flights and {len(other_flights)} other flights available. there details are:
  """
  for i, flight_data in enumerate(best_flights):
    plain_text = parse_flight_data(flight_data)
    plain_text = f"""
    best flight # {i+1}:
    {plain_text}
    """
    flight_data_parsed += plain_text

  for i, flight_data in enumerate(other_flights):
    plain_text = parse_flight_data(flight_data)
    plain_text = f"""
    other flight # {i+1}:
    {plain_text}
    """
    flight_data_parsed += plain_text
  print(flight_data_parsed)
  return flight_data_parsed

In [ ]:
# Call `count_tokens` to get the input token count (`total_tokens`).
print("total_tokens: ", model_flash.count_tokens(flight_data_parsed))

total_tokens:  total_tokens: 3862



# Baku flight

In [ ]:
api_response_baku ={
   "search_metadata":{
      "id":"66d949ae9819f37296191ae9",
      "status":"Success",
      "json_endpoint":"https://serpapi.com/searches/27f0f34d1f8eaa77/66d949ae9819f37296191ae9.json",
      "created_at":"2024-09-05 06:03:26 UTC",
      "processed_at":"2024-09-05 06:03:26 UTC",
      "google_flights_url":"https://www.google.com/travel/flights?hl=en&gl=pk&curr=USD&tfs=CBwQAhojEgoyMDI0LTA5LTEwKAJgoAZqBwgBEgNLSElyBwgBEgNHWUQaIxIKMjAyNC0wOS0zMCgCYKAGagcIARIDR1lEcgcIARIDS0hJQgYBAQICAwRIAWCIJ3ABmAEB",
      "raw_html_file":"https://serpapi.com/searches/27f0f34d1f8eaa77/66d949ae9819f37296191ae9.html",
      "prettify_html_file":"https://serpapi.com/searches/27f0f34d1f8eaa77/66d949ae9819f37296191ae9.prettify",
      "total_time_taken":3.26
   },
   "search_parameters":{
      "engine":"google_flights",
      "hl":"en",
      "gl":"pk",
      "type":"1",
      "departure_id":"KHI",
      "arrival_id":"GYD",
      "outbound_date":"2024-09-10",
      "return_date":"2024-09-30",
      "travel_class":1,
      "adults":2,
      "children":2,
      "infants_in_seat":1,
      "infants_on_lap":1,
      "stops":2,
      "max_price":"5000",
      "max_duration":800,
      "currency":"USD"
   },
   "best_flights":[
      {
         "flights":[
            {
               "departure_airport":{
                  "name":"Jinnah International Airport",
                  "id":"KHI",
                  "time":"2024-09-10 04:55"
               },
               "arrival_airport":{
                  "name":"Dubai International Airport",
                  "id":"DXB",
                  "time":"2024-09-10 06:10"
               },
               "duration":135,
               "airplane":"Boeing 737MAX 8 Passenger",
               "airline":"flydubai",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/FZ.png",
               "travel_class":"Economy",
               "flight_number":"FZ 332",
               "ticket_also_sold_by":[
                  "Emirates"
               ],
               "legroom":"30 in",
               "extensions":[
                  "Average legroom (30 in)",
                  "In-seat USB outlet",
                  "On-demand video",
                  "Carbon emissions estimate: 486 kg"
               ],
               "overnight":True
            },
            {
               "departure_airport":{
                  "name":"Dubai International Airport",
                  "id":"DXB",
                  "time":"2024-09-10 08:10"
               },
               "arrival_airport":{
                  "name":"Heydar Aliyev International Airport",
                  "id":"GYD",
                  "time":"2024-09-10 11:05"
               },
               "duration":175,
               "airplane":"Boeing 737MAX 8 Passenger",
               "airline":"flydubai",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/FZ.png",
               "travel_class":"Economy",
               "flight_number":"FZ 709",
               "ticket_also_sold_by":[
                  "Emirates"
               ],
               "legroom":"30 in",
               "extensions":[
                  "Average legroom (30 in)",
                  "In-seat USB outlet",
                  "On-demand video",
                  "Carbon emissions estimate: 673 kg"
               ]
            }
         ],
         "layovers":[
            {
               "duration":120,
               "name":"Dubai International Airport",
               "id":"DXB"
            }
         ],
         "total_duration":430,
         "carbon_emissions":{
            "this_flight":1161000,
            "typical_for_this_route":1246000,
            "difference_percent":-7
         },
         "price":2128,
         "type":"Round trip",
         "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/FZ.png",
         "departure_token":"WyJDalJJUlRaV01FaERYM014YVUxQlJGbGplbEZDUnkwdExTMHRMUzB0TFhaMGRHa3hOVUZCUVVGQlIySmFVMkUwVEY5NFNsZEJFZ3RHV2pNek1ueEdXamN3T1JvTENLUCtEQkFDR2dOVlUwUTRISENqL2d3PSIsW1siS0hJIiwiMjAyNC0wOS0xMCIsIkRYQiIsbnVsbCwiRloiLCIzMzIiXSxbIkRYQiIsIjIwMjQtMDktMTAiLCJHWUQiLG51bGwsIkZaIiwiNzA5Il1dXQ=="
      },
      {
         "flights":[
            {
               "departure_airport":{
                  "name":"Jinnah International Airport",
                  "id":"KHI",
                  "time":"2024-09-10 11:30"
               },
               "arrival_airport":{
                  "name":"Dubai International Airport",
                  "id":"DXB",
                  "time":"2024-09-10 12:45"
               },
               "duration":135,
               "airplane":"Boeing 737MAX 8 Passenger",
               "airline":"flydubai",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/FZ.png",
               "travel_class":"Economy",
               "flight_number":"FZ 334",
               "ticket_also_sold_by":[
                  "Emirates"
               ],
               "legroom":"30 in",
               "extensions":[
                  "Average legroom (30 in)",
                  "In-seat USB outlet",
                  "On-demand video",
                  "Carbon emissions estimate: 471 kg"
               ]
            },
            {
               "departure_airport":{
                  "name":"Dubai International Airport",
                  "id":"DXB",
                  "time":"2024-09-10 14:10"
               },
               "arrival_airport":{
                  "name":"Heydar Aliyev International Airport",
                  "id":"GYD",
                  "time":"2024-09-10 17:00"
               },
               "duration":170,
               "airplane":"Boeing 737",
               "airline":"flydubai",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/FZ.png",
               "travel_class":"Economy",
               "flight_number":"FZ 707",
               "ticket_also_sold_by":[
                  "Emirates"
               ],
               "legroom":"30 in",
               "extensions":[
                  "Average legroom (30 in)",
                  "In-seat USB outlet",
                  "On-demand video",
                  "Carbon emissions estimate: 797 kg"
               ]
            }
         ],
         "layovers":[
            {
               "duration":85,
               "name":"Dubai International Airport",
               "id":"DXB"
            }
         ],
         "total_duration":390,
         "carbon_emissions":{
            "this_flight":1268000,
            "typical_for_this_route":1246000,
            "difference_percent":2
         },
         "price":2128,
         "type":"Round trip",
         "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/FZ.png",
         "departure_token":"WyJDalJJUlRaV01FaERYM014YVUxQlJGbGplbEZDUnkwdExTMHRMUzB0TFhaMGRHa3hOVUZCUVVGQlIySmFVMkUwVEY5NFNsZEJFZ3RHV2pNek5IeEdXamN3TnhvTENLUCtEQkFDR2dOVlUwUTRISENqL2d3PSIsW1siS0hJIiwiMjAyNC0wOS0xMCIsIkRYQiIsbnVsbCwiRloiLCIzMzQiXSxbIkRYQiIsIjIwMjQtMDktMTAiLCJHWUQiLG51bGwsIkZaIiwiNzA3Il1dXQ=="
      },
      {
         "flights":[
            {
               "departure_airport":{
                  "name":"Jinnah International Airport",
                  "id":"KHI",
                  "time":"2024-09-10 23:55"
               },
               "arrival_airport":{
                  "name":"Dubai International Airport",
                  "id":"DXB",
                  "time":"2024-09-11 01:10"
               },
               "duration":135,
               "airplane":"Boeing 737",
               "airline":"flydubai",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/FZ.png",
               "travel_class":"Economy",
               "flight_number":"FZ 330",
               "ticket_also_sold_by":[
                  "Emirates"
               ],
               "legroom":"30 in",
               "extensions":[
                  "Average legroom (30 in)",
                  "In-seat USB outlet",
                  "On-demand video",
                  "Carbon emissions estimate: 606 kg"
               ]
            },
            {
               "departure_airport":{
                  "name":"Dubai International Airport",
                  "id":"DXB",
                  "time":"2024-09-11 08:10"
               },
               "arrival_airport":{
                  "name":"Heydar Aliyev International Airport",
                  "id":"GYD",
                  "time":"2024-09-11 11:05"
               },
               "duration":175,
               "airplane":"Boeing 737MAX 8 Passenger",
               "airline":"flydubai",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/FZ.png",
               "travel_class":"Economy",
               "flight_number":"FZ 709",
               "ticket_also_sold_by":[
                  "Emirates"
               ],
               "legroom":"30 in",
               "extensions":[
                  "Average legroom (30 in)",
                  "In-seat USB outlet",
                  "On-demand video",
                  "Carbon emissions estimate: 673 kg"
               ]
            }
         ],
         "layovers":[
            {
               "duration":420,
               "name":"Dubai International Airport",
               "id":"DXB",
               "overnight":True
            }
         ],
         "total_duration":730,
         "carbon_emissions":{
            "this_flight":1280000,
            "typical_for_this_route":1246000,
            "difference_percent":3
         },
         "price":2128,
         "type":"Round trip",
         "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/FZ.png",
         "departure_token":"WyJDalJJUlRaV01FaERYM014YVUxQlJGbGplbEZDUnkwdExTMHRMUzB0TFhaMGRHa3hOVUZCUVVGQlIySmFVMkUwVEY5NFNsZEJFZ3RHV2pNek1IeEdXamN3T1JvTENLUCtEQkFDR2dOVlUwUTRISENqL2d3PSIsW1siS0hJIiwiMjAyNC0wOS0xMCIsIkRYQiIsbnVsbCwiRloiLCIzMzAiXSxbIkRYQiIsIjIwMjQtMDktMTEiLCJHWUQiLG51bGwsIkZaIiwiNzA5Il1dXQ=="
      },
      {
         "flights":[
            {
               "departure_airport":{
                  "name":"Jinnah International Airport",
                  "id":"KHI",
                  "time":"2024-09-10 10:35"
               },
               "arrival_airport":{
                  "name":"Sharjah International Airport",
                  "id":"SHJ",
                  "time":"2024-09-10 11:35"
               },
               "duration":120,
               "airplane":"Airbus A320",
               "airline":"Air Arabia",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/G9.png",
               "travel_class":"Economy",
               "flight_number":"G9 543",
               "legroom":"30 in",
               "extensions":[
                  "Average legroom (30 in)",
                  "Stream media to your device",
                  "Carbon emissions estimate: 571 kg"
               ]
            },
            {
               "departure_airport":{
                  "name":"Sharjah International Airport",
                  "id":"SHJ",
                  "time":"2024-09-10 13:05"
               },
               "arrival_airport":{
                  "name":"Heydar Aliyev International Airport",
                  "id":"GYD",
                  "time":"2024-09-10 15:55"
               },
               "duration":170,
               "airplane":"Airbus A320",
               "airline":"Air Arabia",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/G9.png",
               "travel_class":"Economy",
               "flight_number":"G9 303",
               "legroom":"30 in",
               "extensions":[
                  "Average legroom (30 in)",
                  "Stream media to your device",
                  "Carbon emissions estimate: 763 kg"
               ]
            }
         ],
         "layovers":[
            {
               "duration":90,
               "name":"Sharjah International Airport",
               "id":"SHJ"
            }
         ],
         "total_duration":380,
         "carbon_emissions":{
            "this_flight":1335000,
            "typical_for_this_route":1246000,
            "difference_percent":7
         },
         "price":2618,
         "type":"Round trip",
         "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/G9.png",
         "departure_token":"WyJDalJJUlRaV01FaERYM014YVUxQlJGbGplbEZDUnkwdExTMHRMUzB0TFhaMGRHa3hOVUZCUVVGQlIySmFVMkUwVEY5NFNsZEJFZ3RIT1RVME0zeEhPVE13TXhvTENKRDlEeEFDR2dOVlUwUTRISENRL1E4PSIsW1siS0hJIiwiMjAyNC0wOS0xMCIsIlNISiIsbnVsbCwiRzkiLCI1NDMiXSxbIlNISiIsIjIwMjQtMDktMTAiLCJHWUQiLG51bGwsIkc5IiwiMzAzIl1dXQ=="
      }
   ],
   "other_flights":[
      {
         "flights":[
            {
               "departure_airport":{
                  "name":"Jinnah International Airport",
                  "id":"KHI",
                  "time":"2024-09-10 02:45"
               },
               "arrival_airport":{
                  "name":"Dubai International Airport",
                  "id":"DXB",
                  "time":"2024-09-10 03:45"
               },
               "duration":120,
               "airplane":"Boeing 777",
               "airline":"Emirates",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/EK.png",
               "travel_class":"Economy",
               "flight_number":"EK 607",
               "legroom":"32 in",
               "extensions":[
                  "Above average legroom (32 in)",
                  "Wi-Fi for a fee",
                  "In-seat power & USB outlets",
                  "On-demand video",
                  "Carbon emissions estimate: 628 kg"
               ],
               "overnight":True
            },
            {
               "departure_airport":{
                  "name":"Dubai International Airport",
                  "id":"DXB",
                  "time":"2024-09-10 08:10"
               },
               "arrival_airport":{
                  "name":"Heydar Aliyev International Airport",
                  "id":"GYD",
                  "time":"2024-09-10 11:05"
               },
               "duration":175,
               "airplane":"Boeing 737MAX 8 Passenger",
               "airline":"flydubai",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/FZ.png",
               "travel_class":"Economy",
               "flight_number":"FZ 709",
               "ticket_also_sold_by":[
                  "Emirates"
               ],
               "legroom":"30 in",
               "extensions":[
                  "Average legroom (30 in)",
                  "In-seat USB outlet",
                  "On-demand video",
                  "Carbon emissions estimate: 673 kg"
               ]
            }
         ],
         "layovers":[
            {
               "duration":265,
               "name":"Dubai International Airport",
               "id":"DXB",
               "overnight":True
            }
         ],
         "total_duration":560,
         "carbon_emissions":{
            "this_flight":1302000,
            "typical_for_this_route":1246000,
            "difference_percent":4
         },
         "price":2996,
         "type":"Round trip",
         "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/multi.png",
         "departure_token":"WyJDalJJUlRaV01FaERYM014YVUxQlJGbGplbEZDUnkwdExTMHRMUzB0TFhaMGRHa3hOVUZCUVVGQlIySmFVMkUwVEY5NFNsZEJFZ3RGU3pZd04zeEdXamN3T1JvTENJMmtFaEFDR2dOVlUwUTRISENOcEJJPSIsW1siS0hJIiwiMjAyNC0wOS0xMCIsIkRYQiIsbnVsbCwiRUsiLCI2MDciXSxbIkRYQiIsIjIwMjQtMDktMTAiLCJHWUQiLG51bGwsIkZaIiwiNzA5Il1dXQ=="
      },
      {
         "flights":[
            {
               "departure_airport":{
                  "name":"Jinnah International Airport",
                  "id":"KHI",
                  "time":"2024-09-10 12:15"
               },
               "arrival_airport":{
                  "name":"Dubai International Airport",
                  "id":"DXB",
                  "time":"2024-09-10 13:20"
               },
               "duration":125,
               "airplane":"Boeing 777",
               "airline":"Emirates",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/EK.png",
               "travel_class":"Economy",
               "flight_number":"EK 601",
               "legroom":"32 in",
               "extensions":[
                  "Above average legroom (32 in)",
                  "Wi-Fi for a fee",
                  "In-seat power & USB outlets",
                  "On-demand video",
                  "Carbon emissions estimate: 681 kg"
               ]
            },
            {
               "departure_airport":{
                  "name":"Dubai International Airport",
                  "id":"DXB",
                  "time":"2024-09-10 18:15"
               },
               "arrival_airport":{
                  "name":"Heydar Aliyev International Airport",
                  "id":"GYD",
                  "time":"2024-09-10 21:10"
               },
               "duration":175,
               "airplane":"Boeing 737MAX 8 Passenger",
               "airline":"flydubai",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/FZ.png",
               "travel_class":"Economy",
               "flight_number":"FZ 733",
               "ticket_also_sold_by":[
                  "Emirates"
               ],
               "legroom":"30 in",
               "extensions":[
                  "Average legroom (30 in)",
                  "In-seat USB outlet",
                  "On-demand video",
                  "Carbon emissions estimate: 673 kg"
               ]
            }
         ],
         "layovers":[
            {
               "duration":295,
               "name":"Dubai International Airport",
               "id":"DXB"
            }
         ],
         "total_duration":595,
         "carbon_emissions":{
            "this_flight":1355000,
            "typical_for_this_route":1246000,
            "difference_percent":9
         },
         "price":2996,
         "type":"Round trip",
         "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/multi.png",
         "departure_token":"WyJDalJJUlRaV01FaERYM014YVUxQlJGbGplbEZDUnkwdExTMHRMUzB0TFhaMGRHa3hOVUZCUVVGQlIySmFVMkUwVEY5NFNsZEJFZ3RGU3pZd01YeEdXamN6TXhvTENJMmtFaEFDR2dOVlUwUTRISENOcEJJPSIsW1siS0hJIiwiMjAyNC0wOS0xMCIsIkRYQiIsbnVsbCwiRUsiLCI2MDEiXSxbIkRYQiIsIjIwMjQtMDktMTAiLCJHWUQiLG51bGwsIkZaIiwiNzMzIl1dXQ=="
      },
      {
         "flights":[
            {
               "departure_airport":{
                  "name":"Jinnah International Airport",
                  "id":"KHI",
                  "time":"2024-09-10 22:55"
               },
               "arrival_airport":{
                  "name":"Dubai International Airport",
                  "id":"DXB",
                  "time":"2024-09-11 00:10"
               },
               "duration":135,
               "airplane":"Boeing 777",
               "airline":"Emirates",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/EK.png",
               "travel_class":"Economy",
               "flight_number":"EK 603",
               "legroom":"32 in",
               "extensions":[
                  "Above average legroom (32 in)",
                  "Wi-Fi for a fee",
                  "In-seat power & USB outlets",
                  "On-demand video",
                  "Carbon emissions estimate: 689 kg"
               ]
            },
            {
               "departure_airport":{
                  "name":"Dubai International Airport",
                  "id":"DXB",
                  "time":"2024-09-11 08:10"
               },
               "arrival_airport":{
                  "name":"Heydar Aliyev International Airport",
                  "id":"GYD",
                  "time":"2024-09-11 11:05"
               },
               "duration":175,
               "airplane":"Boeing 737MAX 8 Passenger",
               "airline":"flydubai",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/FZ.png",
               "travel_class":"Economy",
               "flight_number":"FZ 709",
               "ticket_also_sold_by":[
                  "Emirates"
               ],
               "legroom":"30 in",
               "extensions":[
                  "Average legroom (30 in)",
                  "In-seat USB outlet",
                  "On-demand video",
                  "Carbon emissions estimate: 673 kg"
               ]
            }
         ],
         "layovers":[
            {
               "duration":480,
               "name":"Dubai International Airport",
               "id":"DXB",
               "overnight":True
            }
         ],
         "total_duration":790,
         "carbon_emissions":{
            "this_flight":1364000,
            "typical_for_this_route":1246000,
            "difference_percent":9
         },
         "price":2996,
         "type":"Round trip",
         "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/multi.png",
         "departure_token":"WyJDalJJUlRaV01FaERYM014YVUxQlJGbGplbEZDUnkwdExTMHRMUzB0TFhaMGRHa3hOVUZCUVVGQlIySmFVMkUwVEY5NFNsZEJFZ3RGU3pZd00zeEdXamN3T1JvTENJMmtFaEFDR2dOVlUwUTRISENOcEJJPSIsW1siS0hJIiwiMjAyNC0wOS0xMCIsIkRYQiIsbnVsbCwiRUsiLCI2MDMiXSxbIkRYQiIsIjIwMjQtMDktMTEiLCJHWUQiLG51bGwsIkZaIiwiNzA5Il1dXQ=="
      },
      {
         "flights":[
            {
               "departure_airport":{
                  "name":"Jinnah International Airport",
                  "id":"KHI",
                  "time":"2024-09-10 04:00"
               },
               "arrival_airport":{
                  "name":"Hamad International Airport",
                  "id":"DOH",
                  "time":"2024-09-10 04:30"
               },
               "duration":150,
               "airplane":"Airbus A350",
               "airline":"Qatar Airways",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/QR.png",
               "travel_class":"Economy",
               "flight_number":"QR 605",
               "legroom":"31 in",
               "extensions":[
                  "Average legroom (31 in)",
                  "Wi-Fi for a fee",
                  "In-seat power & USB outlets",
                  "On-demand video",
                  "Carbon emissions estimate: 648 kg"
               ],
               "overnight":True
            },
            {
               "departure_airport":{
                  "name":"Hamad International Airport",
                  "id":"DOH",
                  "time":"2024-09-10 08:30"
               },
               "arrival_airport":{
                  "name":"Heydar Aliyev International Airport",
                  "id":"GYD",
                  "time":"2024-09-10 12:20"
               },
               "duration":170,
               "airplane":"Airbus A320",
               "airline":"Qatar Airways",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/QR.png",
               "travel_class":"Economy",
               "flight_number":"QR 353",
               "legroom":"31 in",
               "extensions":[
                  "Average legroom (31 in)",
                  "In-seat power & USB outlets",
                  "On-demand video",
                  "Carbon emissions estimate: 937 kg"
               ]
            }
         ],
         "layovers":[
            {
               "duration":240,
               "name":"Hamad International Airport",
               "id":"DOH",
               "overnight":True
            }
         ],
         "total_duration":560,
         "carbon_emissions":{
            "this_flight":1586000,
            "typical_for_this_route":1246000,
            "difference_percent":27
         },
         "price":4568,
         "type":"Round trip",
         "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/QR.png",
         "departure_token":"WyJDalJJUlRaV01FaERYM014YVUxQlJGbGplbEZDUnkwdExTMHRMUzB0TFhaMGRHa3hOVUZCUVVGQlIySmFVMkUwVEY5NFNsZEJFZ3RSVWpZd05YeFJVak0xTXhvTENNendHeEFDR2dOVlUwUTRISERNOEJzPSIsW1siS0hJIiwiMjAyNC0wOS0xMCIsIkRPSCIsbnVsbCwiUVIiLCI2MDUiXSxbIkRPSCIsIjIwMjQtMDktMTAiLCJHWUQiLG51bGwsIlFSIiwiMzUzIl1dXQ=="
      }
   ]
}

In [ ]:
best_flights_baku= api_response_baku["best_flights"]
other_flights_baku= api_response_baku["other_flights"]

In [ ]:
best_flights_baku

[{'flights': [{'departure_airport': {'name': 'Jinnah International Airport',
     'id': 'KHI',
     'time': '2024-09-10 04:55'},
    'arrival_airport': {'name': 'Dubai International Airport',
     'id': 'DXB',
     'time': '2024-09-10 06:10'},
    'duration': 135,
    'airplane': 'Boeing 737MAX 8 Passenger',
    'airline': 'flydubai',
    'airline_logo': 'https://www.gstatic.com/flights/airline_logos/70px/FZ.png',
    'travel_class': 'Economy',
    'flight_number': 'FZ 332',
    'ticket_also_sold_by': ['Emirates'],
    'legroom': '30 in',
    'extensions': ['Average legroom (30 in)',
     'In-seat USB outlet',
     'On-demand video',
     'Carbon emissions estimate: 486 kg'],
    'overnight': True},
   {'departure_airport': {'name': 'Dubai International Airport',
     'id': 'DXB',
     'time': '2024-09-10 08:10'},
    'arrival_airport': {'name': 'Heydar Aliyev International Airport',
     'id': 'GYD',
     'time': '2024-09-10 11:05'},
    'duration': 175,
    'airplane': 'Boeing 737MAX

In [ ]:
parse_flight_data_baku = parse_flight_data_formater(best_flights_baku, other_flights_baku)


  There are 4 best flights and 4 other flights available. there details are:
  
    best flight # 1:
    
                      Flight Number: FZ 332
                      Airline: flydubai
                      Airplane: Boeing 737MAX 8 Passenger
                      Class: Economy
                      Legroom: 30 in
                      Duration: 135 minutes
                      Departure: Jinnah International Airport (KHI) at 2024-09-10 04:55
                      Arrival: Dubai International Airport (DXB) at 2024-09-10 06:10
                      Extensions: Average legroom (30 in), In-seat USB outlet, On-demand video, Carbon emissions estimate: 486 kg
                      This flight is overnight.


                      Flight Number: FZ 709
                      Airline: flydubai
                      Airplane: Boeing 737MAX 8 Passenger
                      Class: Economy
                      Legroom: 30 in
                      Duration: 175 minutes
                    

In [ ]:
print("total_tokens: ", model_flash.count_tokens(parse_flight_data_baku))

total_tokens:  total_tokens: 3448



In [ ]:
prompt_baku = """
I am planning to make a 7 days family trip to Baku,Azerbaijan from karachi,pakistan from 10 of september
my family include me, my wife, a 8 years old son, a daughter of 6 years and a infant of 1.5 years old,
the flight can have max one stop . i prefereed economy class with max budget about $3500.
it will be round trip.
Task:
1- help me to provide a flight plan for my trip to istanbul.
Instruction:
1- user may input city name or airport name, convert it into respective IATA airport code is an uppercase 3-letter code.
e.g for Istanbul airport code the code is "IST" and for Dubai Internationa Airport code is "DXB"
2- if user provide max flight duration, convert it into minutes and if does not proivde max flight duration, set it to 1440 minutes
3- map flight_type as per dictionary  : {2: "one_way",1: "round_trip"}
4- map travel_class as per dictionary : {1: "Economy",2: "premimum Economy",3: "business",4: "first"}
5- map max_stops as per dictionary    : {0: "Any number of stops (default)",1: "Nonstop only",2: "1 stop or fewer",3: "2 stops or fewer}
  """

In [ ]:
def flight_data_baku(departure_id:str,
                arrival_id: str,
                currency: str ,
                outbound_date: str,     #The format is YYYY-MM-DD. e.g. 2024-08-20
                return_date: str,       # The format is YYYY-MM-DD. e.g. 2024-08-20
                flight_type: str,       # flight_type = {"one_way":2,"round_trip":1}
                travel_class: str,      # travel class = {"Economy":1,"premimum Economy":2,"business":3,"first":4}
                adults: int,
                children: int,
                infants: int,
                max_price: int,
                max_flight_duration: int,    # in minutes, like 600 for 10 hours
                max_stops: int    # stops  = {0,Any number of stops (default),1 - Nonstop only,2 - 1 stop or fewer,3 - 2 stops or fewer}
                ):

  """
  This function use Google flight api to fetch the realtime flight data.
  This function requires
  departure city or airport name with expected data,
  arrival city or airport name,
  expected return date or number of days to stay,
  currency type, default is USD,
  travel class e.g: economy, bussines, first,
  flight type e.g : one way, round trip,
  max price in provided currency type,
  max travel duration in minutes,
  max number of stops,
  number of adults,
  number of children,
  number of infants with age,

  This function returns the flight data in json format with details of:
  best flights and other flights(if available)
  flight data contains:
  departure and arrival airport detail like:name, id, time
  duration in minutes,
  airline detail like: name, logo, number
  layovers detail if applicable like : duration, overnight
  price in requested currency
  """
  baku_data = f"""
  There are 4 best flights and 4 other flights available. there details are:

    best flight # 1:

                      Flight Number: FZ 332
                      Airline: flydubai
                      Airplane: Boeing 737MAX 8 Passenger
                      Class: Economy
                      Legroom: 30 in
                      Duration: 135 minutes
                      Departure: Jinnah International Airport (KHI) at 2024-09-10 04:55
                      Arrival: Dubai International Airport (DXB) at 2024-09-10 06:10
                      Extensions: Average legroom (30 in), In-seat USB outlet, On-demand video, Carbon emissions estimate: 486 kg
                      This flight is overnight.


                      Flight Number: FZ 709
                      Airline: flydubai
                      Airplane: Boeing 737MAX 8 Passenger
                      Class: Economy
                      Legroom: 30 in
                      Duration: 175 minutes
                      Departure: Dubai International Airport (DXB) at 2024-09-10 08:10
                      Arrival: Heydar Aliyev International Airport (GYD) at 2024-09-10 11:05
                      Extensions: Average legroom (30 in), In-seat USB outlet, On-demand video, Carbon emissions estimate: 673 kg


        Total Trip Duration: 430 minutes
        Carbon Emissions: 1161000 kg (Typical for this route: 1246000 kg, Difference: -7%)
        Total Price: $2128
        Trip Type: Round trip


    best flight # 2:

                      Flight Number: FZ 334
                      Airline: flydubai
                      Airplane: Boeing 737MAX 8 Passenger
                      Class: Economy
                      Legroom: 30 in
                      Duration: 135 minutes
                      Departure: Jinnah International Airport (KHI) at 2024-09-10 11:30
                      Arrival: Dubai International Airport (DXB) at 2024-09-10 12:45
                      Extensions: Average legroom (30 in), In-seat USB outlet, On-demand video, Carbon emissions estimate: 471 kg


                      Flight Number: FZ 707
                      Airline: flydubai
                      Airplane: Boeing 737
                      Class: Economy
                      Legroom: 30 in
                      Duration: 170 minutes
                      Departure: Dubai International Airport (DXB) at 2024-09-10 14:10
                      Arrival: Heydar Aliyev International Airport (GYD) at 2024-09-10 17:00
                      Extensions: Average legroom (30 in), In-seat USB outlet, On-demand video, Carbon emissions estimate: 797 kg


        Total Trip Duration: 390 minutes
        Carbon Emissions: 1268000 kg (Typical for this route: 1246000 kg, Difference: 2%)
        Total Price: $2128
        Trip Type: Round trip


    best flight # 3:

                      Flight Number: FZ 330
                      Airline: flydubai
                      Airplane: Boeing 737
                      Class: Economy
                      Legroom: 30 in
                      Duration: 135 minutes
                      Departure: Jinnah International Airport (KHI) at 2024-09-10 23:55
                      Arrival: Dubai International Airport (DXB) at 2024-09-11 01:10
                      Extensions: Average legroom (30 in), In-seat USB outlet, On-demand video, Carbon emissions estimate: 606 kg


                      Flight Number: FZ 709
                      Airline: flydubai
                      Airplane: Boeing 737MAX 8 Passenger
                      Class: Economy
                      Legroom: 30 in
                      Duration: 175 minutes
                      Departure: Dubai International Airport (DXB) at 2024-09-11 08:10
                      Arrival: Heydar Aliyev International Airport (GYD) at 2024-09-11 11:05
                      Extensions: Average legroom (30 in), In-seat USB outlet, On-demand video, Carbon emissions estimate: 673 kg


        Total Trip Duration: 730 minutes
        Carbon Emissions: 1280000 kg (Typical for this route: 1246000 kg, Difference: 3%)
        Total Price: $2128
        Trip Type: Round trip


    best flight # 4:

                      Flight Number: G9 543
                      Airline: Air Arabia
                      Airplane: Airbus A320
                      Class: Economy
                      Legroom: 30 in
                      Duration: 120 minutes
                      Departure: Jinnah International Airport (KHI) at 2024-09-10 10:35
                      Arrival: Sharjah International Airport (SHJ) at 2024-09-10 11:35
                      Extensions: Average legroom (30 in), Stream media to your device, Carbon emissions estimate: 571 kg


                      Flight Number: G9 303
                      Airline: Air Arabia
                      Airplane: Airbus A320
                      Class: Economy
                      Legroom: 30 in
                      Duration: 170 minutes
                      Departure: Sharjah International Airport (SHJ) at 2024-09-10 13:05
                      Arrival: Heydar Aliyev International Airport (GYD) at 2024-09-10 15:55
                      Extensions: Average legroom (30 in), Stream media to your device, Carbon emissions estimate: 763 kg


        Total Trip Duration: 380 minutes
        Carbon Emissions: 1335000 kg (Typical for this route: 1246000 kg, Difference: 7%)
        Total Price: $2618
        Trip Type: Round trip


    other flight # 1:

                      Flight Number: EK 607
                      Airline: Emirates
                      Airplane: Boeing 777
                      Class: Economy
                      Legroom: 32 in
                      Duration: 120 minutes
                      Departure: Jinnah International Airport (KHI) at 2024-09-10 02:45
                      Arrival: Dubai International Airport (DXB) at 2024-09-10 03:45
                      Extensions: Above average legroom (32 in), Wi-Fi for a fee, In-seat power & USB outlets, On-demand video, Carbon emissions estimate: 628 kg
                      This flight is overnight.


                      Flight Number: FZ 709
                      Airline: flydubai
                      Airplane: Boeing 737MAX 8 Passenger
                      Class: Economy
                      Legroom: 30 in
                      Duration: 175 minutes
                      Departure: Dubai International Airport (DXB) at 2024-09-10 08:10
                      Arrival: Heydar Aliyev International Airport (GYD) at 2024-09-10 11:05
                      Extensions: Average legroom (30 in), In-seat USB outlet, On-demand video, Carbon emissions estimate: 673 kg


        Total Trip Duration: 560 minutes
        Carbon Emissions: 1302000 kg (Typical for this route: 1246000 kg, Difference: 4%)
        Total Price: $2996
        Trip Type: Round trip


    other flight # 2:

                      Flight Number: EK 601
                      Airline: Emirates
                      Airplane: Boeing 777
                      Class: Economy
                      Legroom: 32 in
                      Duration: 125 minutes
                      Departure: Jinnah International Airport (KHI) at 2024-09-10 12:15
                      Arrival: Dubai International Airport (DXB) at 2024-09-10 13:20
                      Extensions: Above average legroom (32 in), Wi-Fi for a fee, In-seat power & USB outlets, On-demand video, Carbon emissions estimate: 681 kg


                      Flight Number: FZ 733
                      Airline: flydubai
                      Airplane: Boeing 737MAX 8 Passenger
                      Class: Economy
                      Legroom: 30 in
                      Duration: 175 minutes
                      Departure: Dubai International Airport (DXB) at 2024-09-10 18:15
                      Arrival: Heydar Aliyev International Airport (GYD) at 2024-09-10 21:10
                      Extensions: Average legroom (30 in), In-seat USB outlet, On-demand video, Carbon emissions estimate: 673 kg


        Total Trip Duration: 595 minutes
        Carbon Emissions: 1355000 kg (Typical for this route: 1246000 kg, Difference: 9%)
        Total Price: $2996
        Trip Type: Round trip


    other flight # 3:

                      Flight Number: EK 603
                      Airline: Emirates
                      Airplane: Boeing 777
                      Class: Economy
                      Legroom: 32 in
                      Duration: 135 minutes
                      Departure: Jinnah International Airport (KHI) at 2024-09-10 22:55
                      Arrival: Dubai International Airport (DXB) at 2024-09-11 00:10
                      Extensions: Above average legroom (32 in), Wi-Fi for a fee, In-seat power & USB outlets, On-demand video, Carbon emissions estimate: 689 kg


                      Flight Number: FZ 709
                      Airline: flydubai
                      Airplane: Boeing 737MAX 8 Passenger
                      Class: Economy
                      Legroom: 30 in
                      Duration: 175 minutes
                      Departure: Dubai International Airport (DXB) at 2024-09-11 08:10
                      Arrival: Heydar Aliyev International Airport (GYD) at 2024-09-11 11:05
                      Extensions: Average legroom (30 in), In-seat USB outlet, On-demand video, Carbon emissions estimate: 673 kg


        Total Trip Duration: 790 minutes
        Carbon Emissions: 1364000 kg (Typical for this route: 1246000 kg, Difference: 9%)
        Total Price: $2996
        Trip Type: Round trip


    other flight # 4:

                      Flight Number: QR 605
                      Airline: Qatar Airways
                      Airplane: Airbus A350
                      Class: Economy
                      Legroom: 31 in
                      Duration: 150 minutes
                      Departure: Jinnah International Airport (KHI) at 2024-09-10 04:00
                      Arrival: Hamad International Airport (DOH) at 2024-09-10 04:30
                      Extensions: Average legroom (31 in), Wi-Fi for a fee, In-seat power & USB outlets, On-demand video, Carbon emissions estimate: 648 kg
                      This flight is overnight.


                      Flight Number: QR 353
                      Airline: Qatar Airways
                      Airplane: Airbus A320
                      Class: Economy
                      Legroom: 31 in
                      Duration: 170 minutes
                      Departure: Hamad International Airport (DOH) at 2024-09-10 08:30
                      Arrival: Heydar Aliyev International Airport (GYD) at 2024-09-10 12:20
                      Extensions: Average legroom (31 in), In-seat power & USB outlets, On-demand video, Carbon emissions estimate: 937 kg


        Total Trip Duration: 560 minutes
        Carbon Emissions: 1586000 kg (Typical for this route: 1246000 kg, Difference: 27%)
        Total Price: $4568
        Trip Type: Round trip

  """

  return baku_data

In [ ]:
available_functions_baku = {
    "flight_data_baku": flight_data_baku,
}

In [ ]:
instruction = """You are trip planner agent and help to make a trip by taking flight data
from function and analyze the function response and make a plan as per user requirment.
follow the guideline provided in prompt.
"""
model_name = "gemini-1.5-flash"
# model_name = "gemini-1.0-pro"

model_flash_baku = genai.GenerativeModel(
                              model_name=model_name,
                              tools=available_functions_baku.values(),
                              system_instruction=instruction
)
model_flash_baku

genai.GenerativeModel(
    model_name='models/gemini-1.5-flash',
    generation_config={},
    safety_settings={},
    tools=<google.generativeai.types.content_types.FunctionLibrary object at 0x7fa7860baa40>,
    system_instruction='You are trip planner agent and help to make a trip by taking flight data\nfrom function and analyze the function response and make a plan as per user requirment.\nfollow the guideline provided in prompt.\n',
    cached_content=None
)

In [ ]:
response_flash_baku = model_flash_baku.generate_content(prompt_baku)
response_flash_baku

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "flight_data_baku",
                  "args": {
                    "flight_type": "round_trip",
                    "max_flight_duration": 1440.0,
                    "currency": "USD",
                    "max_stops": 2.0,
                    "max_price": 3500.0,
                    "infants": 1.0,
                    "adults": 2.0,
                    "departure_id": "KHI",
                    "travel_class": "Economy",
                    "children": 2.0,
                    "outbound_date": "2023-09-10",
                    "return_date": "2023-09-17",
                    "arrival_id": "GYD"
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_

In [ ]:
from ast import arguments
from datetime import datetime,timedelta
now = datetime.now()
default_outbound_date = now.strftime('%Y-%m-%d')
# Convert string to datetime object
date_object = datetime.strptime(default_outbound_date, '%Y-%m-%d')
# Add 7 days
new_date = date_object + timedelta(days=7)
default_return_date = new_date.strftime('%Y-%m-%d')

for message in response_flash_baku.candidates:
  # print(message.content)
  for part in message.content.parts:
    # print(part)
    try:
      text_content = part.text
      print("Got text")
    except ValueError:
      print("value error text")
    if hasattr(part, 'function_call') and part.function_call is not None:
      print("Got function call")
      try:
        function_call = part.function_call
        if function_call.name in available_functions_baku:
          function_to_call = available_functions_baku[function_call.name]
          print(f"Function {function_call.name} is avalibale")
        else:
          print(f"Function {function_call.name} is not avalibale")

        # Access Arguments
        if hasattr(function_call, "args"):
          print("arguments are present")
          function_arguments = function_call.args
          #Initialized to default values
          departure_id    = "KHI"
          arrival_id      = "DXB"
          currency        = "USD"
          outbound_date   =   default_outbound_date  #The format is YYYY-MM-DD. e.g. 2024-08-20
          return_date     =   default_return_date   # The format is YYYY-MM-DD. e.g. 2024-08-20
          flight_type     =    "1"   # flight_type = {"one_way"2="round_trip"1}
          travel_class    =    1  # travel class = {"Economy"1="premimum Economy"2="business"3="first"4}
          adults          =   1
          children        =   0
          infants         =   0
          max_price       =   "5000"
          max_flight_duration =  1440  # in minutes for e.g 600 for 10 hours
          max_stops       =   0

          # Iterate over each item in 'args'
          for key,value in function_arguments.items():
            if key == "children":
              try:
                children = value
              except Exception as e:
                print(f"Error getting 'children': {e}")
                pass
            if key == "flight_type":
              try:
                flight_type = value
              except Exception as e:
                print(f"Error getting 'flight_type': {e}")
                pass
            if key == "infants":
              try:
                infants = value
              except Exception as e:
                print(f"Error getting 'infants': {e}")
                pass
            if key == "travel_class":
              try:
                travel_class = value
              except Exception as e:
                print(f"Error getting 'travel_class': {e}")
                pass
            if key == "adults":
              try:
                adults = value
              except Exception as e:
                print(f"Error getting 'adults': {e}")
                pass
            if key == "max_flight_duration":
              try:
                max_flight_duration = value
              except Exception as e:
                print(f"Error getting 'max_flight_duration': {e}")
                pass
            if key == "return_date":
              try:
                return_date = value
              except Exception as e:
                print(f"Error getting 'return_date': {e}")
                pass
            if key == "outbound_date":
              try:
                outbound_date = value
              except Exception as e:
                print(f"Error getting 'outbound_date': {e}")
                pass
            if key == "arrival_id":
              try:
                arrival_id = value
              except Exception as e:
                print(f"Error getting 'arrival_id': {e}")
                pass
            if key == "departure_id":
              try:
                departure_id = value
              except Exception as e:
                print(f"Error getting 'departure_id': {e}")
                pass
            if key == "currency":
              try:
                currency = value
              except Exception as e:
                print(f"Error getting 'currency': {e}")
                pass
            if key == "max_price":
              try:
                max_price = value
              except Exception as e:
                print(f"Error getting 'max_price': {e}")
                pass
            if key == "max_stops":
              try:
                max_stops = value
              except Exception as e:
                print(f"Error getting 'max_stops': {e}")
                pass

        #calling google api
        flight_api_response_baku = flight_data_baku(
          departure_id    ,
          arrival_id      ,
          currency        ,
          outbound_date   ,
          return_date     ,
          flight_type     ,
          travel_class    ,
          adults          ,
          children        ,
          infants         ,
          max_price       ,
          max_flight_duration,
          max_stops       , )

        list_content_baku = [prompt_baku, flight_api_response_baku,f"Now continue from last message and helop plan travel in text formt - Funcation Calling completed."]

        second_response_baku = model_flash_baku.generate_content(list_content_baku)

        for message in second_response_baku.candidates:
          print("message.content", message.content)
          for part in message.content.parts:
            print("part", part)
            try:
              text_content = part.text
              print("Got text")
              print("text_content", text_content)
            except ValueError:
              print("value error text")
              pass

      except ValueError:
        print("value error function call")

Got text
Got function call
Function flight_data_baku is not avalibale
arguments are present
message.content parts {
  text: "Okay, I see. Based on your requirements, here\'s a possible flight plan for your family trip to Baku, Azerbaijan from Karachi, Pakistan:\n\n**Flight Details:**\n\n* **Departure:** Karachi, Pakistan (KHI)\n* **Arrival:** Baku, Azerbaijan (GYD)\n* **Date:** September 10, 2024\n* **Return:** September 17, 2024\n* **Flight Type:** Round Trip\n* **Travel Class:** Economy\n* **Maximum Stops:** 1\n* **Maximum Budget:** $3500\n\n**Recommended Flight:**\n\n* **Best Flight #1:** Flydubai FZ 332 (KHI to DXB) departing at 4:55 AM on September 10, 2024, followed by Flydubai FZ 709 (DXB to GYD) departing at 8:10 AM on September 10, 2024. \n* **Total Trip Duration:** 430 minutes\n* **Total Price:** $2128\n* **Carbon Emissions:** 1161000 kg \n\n**Explanation:**\n\nThis flight plan utilizes flydubai, providing a convenient connection in Dubai and staying within your budget of $35

In [ ]:
final_response_baku = f"""Okay, I see. Based on your requirements, here's a possible flight plan for your family trip to Baku, Azerbaijan from Karachi, Pakistan:

**Flight Details:**

* **Departure:** Karachi, Pakistan (KHI)
* **Arrival:** Baku, Azerbaijan (GYD)
* **Date:** September 10, 2024
* **Return:** September 17, 2024
* **Flight Type:** Round Trip
* **Travel Class:** Economy
* **Maximum Stops:** 1
* **Maximum Budget:** $3500

**Recommended Flight:**

* **Best Flight #1:** Flydubai FZ 332 (KHI to DXB) departing at 4:55 AM on September 10, 2024, followed by Flydubai FZ 709 (DXB to GYD) departing at 8:10 AM on September 10, 2024.
* **Total Trip Duration:** 430 minutes
* **Total Price:** $2128
* **Carbon Emissions:** 1161000 kg

**Explanation:**

This flight plan utilizes flydubai, providing a convenient connection in Dubai and staying within your budget of $3500. The flight times are reasonable and the total trip duration is under 7 hours.

**Things to Consider:**

* **Family Needs:**  Remember to factor in the needs of your family, especially the infant, when making final decisions.
* **Connecting Flights:** Be sure to check the layover time at Dubai International Airport (DXB) to ensure it is sufficient for your family, including time for baggage collection and security.
* **Flexibility:**  Flight prices fluctuate, so you may find slightly different options if you have flexibility with your travel dates.

**Next Steps:**

* **Booking:** You can use a flight booking website or app to book your flights once you have chosen the best option for you.
* **Accommodation:**  Start looking for accommodations in Baku that suit your family's needs and budget.
* **Visa Requirements:** If needed, ensure you understand the visa requirements for Azerbaijan and apply for any necessary visas in advance.
* **Travel Insurance:** Purchase travel insurance to protect yourself and your family in case of any unforeseen events.
"""

In [ ]:
Markdown(final_response_baku)

Okay, I see. Based on your requirements, here's a possible flight plan for your family trip to Baku, Azerbaijan from Karachi, Pakistan:

**Flight Details:**

* **Departure:** Karachi, Pakistan (KHI)
* **Arrival:** Baku, Azerbaijan (GYD)
* **Date:** September 10, 2024
* **Return:** September 17, 2024
* **Flight Type:** Round Trip
* **Travel Class:** Economy
* **Maximum Stops:** 1
* **Maximum Budget:** $3500

**Recommended Flight:**

* **Best Flight #1:** Flydubai FZ 332 (KHI to DXB) departing at 4:55 AM on September 10, 2024, followed by Flydubai FZ 709 (DXB to GYD) departing at 8:10 AM on September 10, 2024. 
* **Total Trip Duration:** 430 minutes
* **Total Price:** $2128
* **Carbon Emissions:** 1161000 kg 

**Explanation:**

This flight plan utilizes flydubai, providing a convenient connection in Dubai and staying within your budget of $3500. The flight times are reasonable and the total trip duration is under 7 hours. 

**Things to Consider:**

* **Family Needs:**  Remember to factor in the needs of your family, especially the infant, when making final decisions.  
* **Connecting Flights:** Be sure to check the layover time at Dubai International Airport (DXB) to ensure it is sufficient for your family, including time for baggage collection and security. 
* **Flexibility:**  Flight prices fluctuate, so you may find slightly different options if you have flexibility with your travel dates. 

**Next Steps:**

* **Booking:** You can use a flight booking website or app to book your flights once you have chosen the best option for you. 
* **Accommodation:**  Start looking for accommodations in Baku that suit your family's needs and budget.
* **Visa Requirements:** If needed, ensure you understand the visa requirements for Azerbaijan and apply for any necessary visas in advance.
* **Travel Insurance:** Purchase travel insurance to protect yourself and your family in case of any unforeseen events. 


In [ ]:
instruction = """You are trip planner agent and help to make a trip by taking flight data
from function and analyze the function response and make a plan as per user requirment.

"""
# model_name = "gemini-1.5-flash"
model_name = "gemini-1.5-pro"

model_pro_baku = genai.GenerativeModel(
                              model_name=model_name,
                              tools=available_functions_baku.values(),
                              system_instruction=instruction
)
model_pro_baku

genai.GenerativeModel(
    model_name='models/gemini-1.5-pro',
    generation_config={},
    safety_settings={},
    tools=<google.generativeai.types.content_types.FunctionLibrary object at 0x7a64c569c9d0>,
    system_instruction='You are trip planner agent and help to make a trip by taking flight data\nfrom function and analyze the function response and make a plan as per user requirment.\n\n',
    cached_content=None
)

In [ ]:
response_pro_baku = model_pro_baku.generate_content(prompt_baku)

In [ ]:
from ast import arguments
from datetime import datetime,timedelta
now = datetime.now()
default_outbound_date = now.strftime('%Y-%m-%d')
# Convert string to datetime object
date_object = datetime.strptime(default_outbound_date, '%Y-%m-%d')
# Add 7 days
new_date = date_object + timedelta(days=7)
default_return_date = new_date.strftime('%Y-%m-%d')

for message in response_pro_baku.candidates:
  # print(message.content)
  for part in message.content.parts:
    # print(part)
    try:
      text_content = part.text
      print("Got text")
    except ValueError:
      print("value error text")
    if hasattr(part, 'function_call') and part.function_call is not None:
      print("Got function call")
      try:
        function_call = part.function_call
        if function_call.name in available_functions_baku:
          function_to_call = available_functions_baku[function_call.name]
          print(f"Function {function_call.name} is avalibale")
        else:
          print(f"Function {function_call.name} is not avalibale")

        # Access Arguments
        if hasattr(function_call, "args"):
          print("arguments are present")
          function_arguments = function_call.args
          #Initialized to default values
          departure_id    = "KHI"
          arrival_id      = "DXB"
          currency        = "USD"
          outbound_date   =   default_outbound_date  #The format is YYYY-MM-DD. e.g. 2024-08-20
          return_date     =   default_return_date   # The format is YYYY-MM-DD. e.g. 2024-08-20
          flight_type     =    "1"   # flight_type = {"one_way"2="round_trip"1}
          travel_class    =    1  # travel class = {"Economy"1="premimum Economy"2="business"3="first"4}
          adults          =   1
          children        =   0
          infants         =   0
          max_price       =   "5000"
          max_flight_duration =  1440  # in minutes for e.g 600 for 10 hours
          max_stops       =   0

          # Iterate over each item in 'args'
          for key,value in function_arguments.items():
            if key == "children":
              try:
                children = value
              except Exception as e:
                print(f"Error getting 'children': {e}")
                pass
            if key == "flight_type":
              try:
                flight_type = value
              except Exception as e:
                print(f"Error getting 'flight_type': {e}")
                pass
            if key == "infants":
              try:
                infants = value
              except Exception as e:
                print(f"Error getting 'infants': {e}")
                pass
            if key == "travel_class":
              try:
                travel_class = value
              except Exception as e:
                print(f"Error getting 'travel_class': {e}")
                pass
            if key == "adults":
              try:
                adults = value
              except Exception as e:
                print(f"Error getting 'adults': {e}")
                pass
            if key == "max_flight_duration":
              try:
                max_flight_duration = value
              except Exception as e:
                print(f"Error getting 'max_flight_duration': {e}")
                pass
            if key == "return_date":
              try:
                return_date = value
              except Exception as e:
                print(f"Error getting 'return_date': {e}")
                pass
            if key == "outbound_date":
              try:
                outbound_date = value
              except Exception as e:
                print(f"Error getting 'outbound_date': {e}")
                pass
            if key == "arrival_id":
              try:
                arrival_id = value
              except Exception as e:
                print(f"Error getting 'arrival_id': {e}")
                pass
            if key == "departure_id":
              try:
                departure_id = value
              except Exception as e:
                print(f"Error getting 'departure_id': {e}")
                pass
            if key == "currency":
              try:
                currency = value
              except Exception as e:
                print(f"Error getting 'currency': {e}")
                pass
            if key == "max_price":
              try:
                max_price = value
              except Exception as e:
                print(f"Error getting 'max_price': {e}")
                pass
            if key == "max_stops":
              try:
                max_stops = value
              except Exception as e:
                print(f"Error getting 'max_stops': {e}")
                pass

        #calling google api
        flight_api_response_baku = flight_data_baku(
          departure_id    ,
          arrival_id      ,
          currency        ,
          outbound_date   ,
          return_date     ,
          flight_type     ,
          travel_class    ,
          adults          ,
          children        ,
          infants         ,
          max_price       ,
          max_flight_duration,
          max_stops       , )

        list_content_baku = [prompt_baku, flight_api_response_baku,f"Now continue from last message and helop plan travel in text formt - Funcation Calling completed."]

        second_response_baku = model_pro_baku.generate_content(list_content_baku)

        for message in second_response_baku.candidates:
          print("message.content", message.content)
          for part in message.content.parts:
            print("part", part)
            try:
              text_content = part.text
              print("Got text")
              print("text_content", text_content)
            except ValueError:
              print("value error text")
              pass

      except ValueError:
        print("value error function call")

Got text
Got function call
Function flight_data_baku is avalibale
arguments are present
message.content parts {
  text: "Based on your requirements I would recommend  **best flight # 1**:\n\n**Karachi (KHI) to Baku (GYD)**\n\n*   **Departure**: Jinnah International Airport (KHI) at 2024-09-10 04:55 \n*   **Arrival**: Dubai International Airport (DXB) at 2024-09-10 06:10\n*   **Flight**: FZ 332 operated by flydubai, Boeing 737MAX 8\n*   **Duration**: 135 minutes\n\n*Layover in Dubai (DXB) - Overnight*\n\n*   **Departure**: Dubai International Airport (DXB) at 2024-09-10 08:10\n*   **Arrival**: Heydar Aliyev International Airport (GYD) at 2024-09-10 11:05\n*   **Flight**: FZ 709 operated by flydubai, Boeing 737MAX 8\n*   **Duration**: 175 minutes \n\n**Total Trip Duration: 430 minutes**\n\n**Total Price: $2128**\n\nThis flight offers the shortest travel time with an overnight layover in Dubai. It also has the lowest carbon emissions among the best flight options.  Let me know if you woul

Automatic fucntion calling
https://github.com/google-gemini/cookbook/blob/main/quickstarts/Function_calling.ipynb

In [ ]:
chat = model_flash_baku.start_chat(enable_automatic_function_calling=True)

In [ ]:
chat

ChatSession(
    model=genai.GenerativeModel(
        model_name='models/gemini-1.5-flash',
        generation_config={},
        safety_settings={},
        tools=<google.generativeai.types.content_types.FunctionLibrary object at 0x7fa7860baa40>,
        system_instruction='You are trip planner agent and help to make a trip by taking flight data\nfrom function and analyze the function response and make a plan as per user requirment.\nfollow the guideline provided in prompt.\n',
        cached_content=None
    ),
    history=[]
)

In [ ]:
response = chat.send_message(prompt_baku)
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "I have found a few flight options for your family trip to Baku, Azerbaijan from Karachi, Pakistan. \n\nThe best option seems to be **Flight #1** with **flydubai** for a total price of **$2128**. This flight includes a layover in Dubai and has a total trip duration of **430 minutes**. \n\nLet me know if you have any other preferences, like specific airlines or flight times. I can help you find other options based on your needs. \n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "H

In [ ]:
Markdown(response.candidates[0].content.parts[0].text)

I have found a few flight options for your family trip to Baku, Azerbaijan from Karachi, Pakistan. 

The best option seems to be **Flight #1** with **flydubai** for a total price of **$2128**. This flight includes a layover in Dubai and has a total trip duration of **430 minutes**. 

Let me know if you have any other preferences, like specific airlines or flight times. I can help you find other options based on your needs. 


https://serpapi.com/playground?engine=google_flights&departure_id=KHI&arrival_id=IST&gl=pk&hl=en&currency=USD&type=1&outbound_date=2024-09-10&return_date=2024-09-30&travel_class=1&adults=2&children=2&infants_in_seat=1&infants_on_lap=1&stops=2&max_price=5000&max_duration=800

In [ ]:
response = chat.send_message("whats the detail of flight#1")
Markdown(response.candidates[0].content.parts[0].text)

Here are the details of Flight #1:

**Outbound Flight:**

* **Flight Number:** FZ 332
* **Airline:** flydubai
* **Airplane:** Boeing 737MAX 8 Passenger
* **Class:** Economy
* **Legroom:** 30 in
* **Duration:** 135 minutes
* **Departure:** Jinnah International Airport (KHI) at 2024-09-10 04:55
* **Arrival:** Dubai International Airport (DXB) at 2024-09-10 06:10
* **Extensions:** Average legroom (30 in), In-seat USB outlet, On-demand video, Carbon emissions estimate: 486 kg
* **Note:** This flight is overnight.

**Inbound Flight:**

* **Flight Number:** FZ 709
* **Airline:** flydubai
* **Airplane:** Boeing 737MAX 8 Passenger
* **Class:** Economy
* **Legroom:** 30 in
* **Duration:** 175 minutes
* **Departure:** Dubai International Airport (DXB) at 2024-09-10 08:10
* **Arrival:** Heydar Aliyev International Airport (GYD) at 2024-09-10 11:05
* **Extensions:** Average legroom (30 in), In-seat USB outlet, On-demand video, Carbon emissions estimate: 673 kg

**Total Trip Details:**

* **Total Trip Duration:** 430 minutes
* **Carbon Emissions:** 1161000 kg (Typical for this route: 1246000 kg, Difference: -7%)
* **Total Price:** $2128
* **Trip Type:** Round trip 


In [ ]:
response = chat.send_message("is there any direct flight, i can increase my budget")
Markdown(response.candidates[0].content.parts[0].text)


Unfortunately, there are no direct flights available within your increased budget of $4500. All the options still require a layover. 

The cheapest option with a layover is still **Flight #1** with flydubai for **$2128**, with a layover in Dubai. 

Would you like to explore other options with higher budgets? Or perhaps consider adjusting your travel dates for more direct flight options? 


In [ ]:
response = chat.send_message("is there any flight with qatar airways, i can increase my budget")
Markdown(response.candidates[0].content.parts[0].text)

I found a flight with Qatar Airways! It's **Flight #4** and has a total price of **$4568**. 

Here are the details:

**Outbound Flight:**

* **Flight Number:** QR 605
* **Airline:** Qatar Airways
* **Airplane:** Airbus A350
* **Class:** Economy
* **Legroom:** 31 in
* **Duration:** 150 minutes
* **Departure:** Jinnah International Airport (KHI) at 2024-09-10 04:00
* **Arrival:** Hamad International Airport (DOH) at 2024-09-10 04:30
* **Extensions:** Average legroom (31 in), Wi-Fi for a fee, In-seat power & USB outlets, On-demand video, Carbon emissions estimate: 648 kg
* **Note:** This flight is overnight.

**Inbound Flight:**

* **Flight Number:** QR 353
* **Airline:** Qatar Airways
* **Airplane:** Airbus A320
* **Class:** Economy
* **Legroom:** 31 in
* **Duration:** 170 minutes
* **Departure:** Hamad International Airport (DOH) at 2024-09-10 08:30
* **Arrival:** Heydar Aliyev International Airport (GYD) at 2024-09-10 12:20
* **Extensions:** Average legroom (31 in), In-seat power & USB outlets, On-demand video, Carbon emissions estimate: 937 kg

**Total Trip Details:**

* **Total Trip Duration:** 560 minutes
* **Carbon Emissions:** 1586000 kg (Typical for this route: 1246000 kg, Difference: 27%)
* **Total Price:** $4568
* **Trip Type:** Round trip

Would you like to know more about this flight, or compare it to other options within your budget? 


In [ ]:
def get_hotel_info(
    location: str,
    check_in_date: str,
    check_out_date: str,
    adults: int,
    children: int,
    currency: str,
    max_price: int,
    hotel_class: int
):
    """
    Fetch hotel information from the Google Hotel API.

    Args:
        location (str):
            The location of the hotel. Can be a city name, address, or coordinates (latitude, longitude).
            Example: "New York", "40.7128,-74.0060"

        check_in_date (str):
            The date for checking in to the hotel. Must be in the format YYYY-MM-DD.
            Example: "2024-09-15"

        check_out_date (str):
            The date for checking out of the hotel. Must be in the format YYYY-MM-DD.
            Example: "2024-09-20"

        adults (int):
            The number of adults staying in the hotel room. Should be a positive integer.
            Example: 2

        children (int):
            The number of children staying in the hotel room. Should be a non-negative integer.
            Example: 1

        currency (str):
            The currency code for pricing, typically a 3-letter ISO 4217 code.
            Example: "USD"

        max_price (int):
            The maximum price (in the specified currency) that you are willing to pay for the hotel. Should be a positive integer.
            Example: 300

        hotel_class (int):
            The class (or star rating) of the hotel. Should be an integer value representing the hotel class (typically from 1 to 5).
            Example: 4

    Returns:
        dict: A dictionary containing details about available hotels matching the specified criteria.
    """
    pass  # Function implementation goes here



In [ ]:
bali_hotels= {
  "search_metadata": {
    "id": "65533314b68f8b5cf35732bd",
    "status": "Success",
    "json_endpoint": "https://serpapi.com/searches/7e9e85e70fe7f094/65533314b68f8b5cf35732bd.json",
    "created_at": "2023-11-14 08:43:00 UTC",
    "processed_at": "2023-11-14 08:43:00 UTC",
    "google_hotels_url": "https://www.google.com/_/TravelFrontendUi/data/batchexecute?rpcids=AtySUc&source-path=/travel/search&hl=en&gl=us&rt=c&soc-app=162&soc-platform=1&soc-device=1",
    "raw_html_file": "https://serpapi.com/searches/7e9e85e70fe7f094/65533314b68f8b5cf35732bd.html",
    "prettify_html_file": "https://serpapi.com/searches/7e9e85e70fe7f094/65533314b68f8b5cf35732bd.prettify",
    "total_time_taken": 1.36
  },
  "search_parameters": {
    "engine": "google_hotels",
    "q": "Bali Resorts",
    "gl": "us",
    "hl": "en",
    "currency": "USD",
    "check_in_date": "2024-09-11",
    "check_out_date": "2024-09-12",
    "adults": 2,
    "children": 0
  },
  "search_information": {
    "total_results": 15000
  },
  "brands": [
    {
      "id": 33,
      "name": "Accor Live Limitless",
      "children": [
        {
          "id": 67,
          "name": "Banyan Tree"
        },
        {
          "id": 101,
          "name": "Grand Mercure"
        },
        {
          "id": 21,
          "name": "Ibis"
        },
        ...
      ]
    },
    {
      "id": 229,
      "name": "Aston"
    },
    {
      "id": 18,
      "name": "Best Western International",
      "children": [
        {
          "id": 155,
          "name": "Best Western"
        },
        {
          "id": 105,
          "name": "Best Western Premier"
        }
      ]
    },
    ...
  ],
  "properties": [
    {
      "type": "hotel",
      "name": "The Ritz-Carlton, Bali",
      "description": "Zen-like quarters, some with butler service, in an upscale property offering refined dining & a spa.",
      "logo": "https://www.gstatic.com/travel-hotels/branding/df3424c9-7799-4d27-aeb5-cdcabf8bf950.png",
      "sponsored": True,
      "gps_coordinates": {
        "latitude": -8.830670999999999,
        "longitude": 115.21533099999999
      },
      "check_in_time": "3:00 PM",
      "check_out_time": "12:00 PM",
      "rate_per_night": {
        "lowest": "$347",
        "extracted_lowest": 347,
        "before_taxes_fees": "$287",
        "extracted_before_taxes_fees": 287
      },
      "prices": [
        {
          "source": "The Ritz-Carlton, Bali",
          "logo": "https://www.gstatic.com/travel-hotels/branding/df3424c9-7799-4d27-aeb5-cdcabf8bf950.png",
          "rate_per_night": {
            "lowest": "$347",
            "extracted_lowest": 347,
            "before_taxes_fees": "$287",
            "extracted_before_taxes_fees": 287
          }
        }
      ],
      "nearby_places": [
        {
          "name": "I Gusti Ngurah Rai International Airport",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "29 min"
            }
          ]
        },
        {
          "name": "Bejana, Indonesian Restaurant at The Ritz-Carlton, Bali",
          "transportations": [
            {
              "type": "Walking",
              "duration": "1 min"
            }
          ]
        }
      ],
      "hotel_class": "5-star hotel",
      "extracted_hotel_class": 5,
      "images": [
        {
          "thumbnail": "https://lh3.googleusercontent.com/proxy/3GU0rF7c5y00MbsWRPAkzdY0Mql0YhH7coFNK9nRDE8GwlzsRbc7xHB8lu8ZN6ApPUiuM7GvjB4RWoJaQCcD4kjARaoyDitH27WhWiAdz8dfG4TCY6pafMo52UQH5W76rgH3JESuGN3ohZ20fEwSgBKGmdQZOA=s287-w287-h192-n-k-no-v1",
          "original_image": "https://d2hyz2bfif3cr8.cloudfront.net/imageRepo/7/0/151/470/91/rz-dpssw-private-pool-29237_Classic-Hor_O.jpg"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/proxy/IUakfiu-4guLHoPdx1ippkGtRxwdDW9pxf3j8kRq8FtIKOnnCepdr1DBB1vDftDvbY1IDqTCsgzrvgXdzBB6sJU8-z-7yawWRg-tsLlqSy9XI9mbudAurUnJBm9tmF4sJJFZXkuiyUc7zaNMZ6XPZ3MJDhEzWw=s287-w287-h192-n-k-no-v1",
          "original_image": "https://d2hyz2bfif3cr8.cloudfront.net/imageRepo/7/0/147/874/299/dpssw-villa-0105-hor-clsc_O.jpg"
        },
        {
          "thumbnail": "https://lh6.googleusercontent.com/proxy/MWhomwXIPhVXgnq1drMsEuPzTr2FB1f5ePbnpGxtyxwOcB0zr7xKrN34pDzJQVEPRJxFtaSFKfGngytJW4jO2c_9jiVCQuOb6OgRBqn-TEAI172VO2Ptx3G1bx1OpnvBsIcIhTk4ZoNH4EWwUIRxEgVoYQg2EQ=s287-w287-h192-n-k-no-v1",
          "original_image": "https://d2hyz2bfif3cr8.cloudfront.net/imageRepo/7/0/147/874/243/dpssw-villa-0107-hor-clsc_O.jpg"
        },

      ],
      "overall_rating": 4.6,
      "reviews": 3614,
      "location_rating": 2.8,
      "reviews_breakdown": [
        {
          "name": "Property",
          "description": "Property",
          "total_mentioned": 605,
          "positive": 534,
          "negative": 44,
          "neutral": 27
        },
        {
          "name": "Service",
          "description": "Service",
          "total_mentioned": 599,
          "positive": 507,
          "negative": 74,
          "neutral": 18
        },
        {
          "name": "Nature",
          "description": "Nature and outdoor activities",
          "total_mentioned": 256,
          "positive": 212,
          "negative": 29,
          "neutral": 15
        }
      ],
      "amenities": [
        "Free Wi-Fi",
        "Free parking",
        "Pools",
        "Hot tub",
        "Air conditioning",
        "Fitness centre",
        "Spa",
        "Beach access",
        "Bar",
        "Restaurant",
        "Room service",
        "Kitchen in some rooms",
        "Airport shuttle",
        "Full-service laundry",
        "Accessible",
        "Business centre",
        "Child-friendly",
        "Smoke-free property"
      ],
      "property_token": "ChcIyo2FjenOuZ8xGgsvZy8xdGYyMTV2aBAB",
      "serpapi_property_details_link": "<SerpApi JSON endpoint>"
    },
    {
      "type": "hotel",
      "name": "Mulia Resort - Nusa Dua, Bali",
      "description": "Upmarket rooms & suites in a premium beachfront getaway with 6 restaurants & 4 pools.",
      "logo": "https://www.gstatic.com/travel-hotels/branding/icon_220.png",
      "sponsored": True,
      "gps_coordinates": {
        "latitude": -8.81621,
        "longitude": 115.22117399999999
      },
      "check_in_time": "3:00 PM",
      "check_out_time": "12:00 PM",
      "rate_per_night": {
        "lowest": "$484",
        "extracted_lowest": 484,
        "before_taxes_fees": "$388",
        "extracted_before_taxes_fees": 388
      },
      "prices": [
        {
          "source": "Priceline",
          "logo": "https://www.gstatic.com/travel-hotels/branding/icon_220.png",
          "rate_per_night": {
            "lowest": "$484",
            "extracted_lowest": 484,
            "before_taxes_fees": "$388",
            "extracted_before_taxes_fees": 388
          }
        }
      ],
      "nearby_places": [
        {
          "name": "I Gusti Ngurah Rai International Airport",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "21 min"
            }
          ]
        },

      ],
      "hotel_class": "5-star hotel",
      "extracted_hotel_class": 5,
      "images": [
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipNP9sahhj7GhCqvn_UuJ6E_k6YLxdVXKgj_L1Pe=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipNP9sahhj7GhCqvn_UuJ6E_k6YLxdVXKgj_L1Pe=s10000"
        },

      ],
      "overall_rating": 4.7,
      "reviews": 7567,
      "location_rating": 3.5,
      "reviews_breakdown": [
        {
          "name": "Property",
          "description": "Property",
          "total_mentioned": 1088,
          "positive": 971,
          "negative": 58,
          "neutral": 59
        },

      ],
      "amenities": [
        "Free breakfast",
        "Free Wi-Fi",
        "Parking ($)",

      ],
      "property_token": "ChcIyo2FjteOuZ8xGgsvZy8xdGYyMTV2aBAB",
      "serpapi_property_details_link": "<SerpApi JSON endpoint>"
    },
    {
      "type": "hotel",
      "name": "Anantara Uluwatu Bali Resort",
      "description": "Chic suites with in-room bars & high-end sound systems, plus restaurants, a spa & an infinity pool.",
      "link": "https://www.anantara.com/uluwatu-bali?utm_source=GoogleMyBusiness&utm_medium=GMB&utm_term=Uluwatu&utm_content=Home&utm_campaign=Google_GMB",
      "gps_coordinates": {
        "latitude": -8.8085355,
        "longitude": 115.107911
      },
      "check_in_time": "3:00 PM",
      "check_out_time": "12:00 PM",
      "rate_per_night": {
        "lowest": "$247",
        "extracted_lowest": 247,
        "before_taxes_fees": "$199",
        "extracted_before_taxes_fees": 199
      },
      "total_rate": {
        "lowest": "$1,233",
        "extracted_lowest": 1233,
        "before_taxes_fees": "$994",
        "extracted_before_taxes_fees": 994
      },
      "deal": "27% less than usual",
      "deal_description": "Great Deal",
      "nearby_places": [
        {
          "name": "Padang Padang Beach",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "10 min"
            }
          ]
        },

      ],
      "hotel_class": "5-star hotel",
      "extracted_hotel_class": 5,
      "images": [
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipOWiZ-Gb1n4TFh_g0UJSqViox2Gt5X30ESdFIg9=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipOWiZ-Gb1n4TFh_g0UJSqViox2Gt5X30ESdFIg9=s10000"
        },

      ],
      "overall_rating": 4.5,
      "reviews": 2210,
      "ratings": [
        {
          "stars": 5,
          "count": 1613
        },
        {
          "stars": 4,
          "count": 350
        },
        {
          "stars": 3,
          "count": 118
        },
        {
          "stars": 2,
          "count": 38
        },
        {
          "stars": 1,
          "count": 91
        }
      ],
      "location_rating": 3.6,
      "reviews_breakdown": [
        {
          "name": "Service",
          "description": "Service",
          "total_mentioned": 397,
          "positive": 324,
          "negative": 53,
          "neutral": 20
        },

      ],
      "amenities": [
        "Breakfast",
        "Free Wi-Fi",
        "Free parking",

      ],
      "property_token": "ChcIyo2FjenOuZ8xGgsvgodxdGYyMTV2aBAB",
      "serpapi_property_details_link": "<SerpApi JSON endpoint>"
    },

  ],
  "serpapi_pagination": {
    "current_from": 1,
    "current_to": 20,
    "next_page_token": "CBI=",
    "next": "https://serpapi.com/search.json?adults=2&check_in_date=2024-09-11&check_out_date=2024-09-12&children=0&currency=USD&engine=google_hotels&gl=us&hl=en&next_page_token=CBI%3D&q=Bali+Resorts"
  }
}

In [ ]:
bali_hotels["properties"][0]["gps_coordinates"]["latitude"]

-8.830670999999999

In [ ]:
# Function to parse hotel data to plain text
def parse_hotel_data(hotel_data):
    plain_text_output = []

    # Basic information
    plain_text_output.append(f"Hotel Name: {hotel_data['name']} \n")
    plain_text_output.append(f"Description: {hotel_data['description']}\n")
    plain_text_output.append(f"Check-in Time: {hotel_data['check_in_time']}\n")
    plain_text_output.append(f"Check-out Time: {hotel_data['check_out_time']}\n")
    plain_text_output.append(f"Hotel Class: {hotel_data['hotel_class']}\n")
    plain_text_output.append(f"Overall Rating: {hotel_data['overall_rating']} (based on {hotel_data['reviews']} reviews)\n")

    #location
    plain_text_output.append(f"GPS coordinates: {hotel_data['gps_coordinates']['latitude']} , {hotel_data['gps_coordinates']['longitude']}\n")

    #images
    plain_text_output.append("\nImages:\n")
    for i,image in enumerate(hotel_data['images']):
        plain_text_output.append(image['original_image'])

    # Pricing
    rate = hotel_data['rate_per_night']
    plain_text_output.append(f"\n Rate per Night (before taxes): {rate['before_taxes_fees']}\n")
    plain_text_output.append(f"Rate per Night (lowest): {rate['lowest']}\n")

    # source of price
    plain_text_output.append("\nSoruces of price :\n")
    for i, source in enumerate(hotel_data['prices']):
        plain_text_output.append(f"{i+1}. {source['source']}\n")


    # Nearby places
    plain_text_output.append("\nNearby Places:")
    for i,place in enumerate(hotel_data['nearby_places']):
        name = place['name']
        transportation = place['transportations'][0]
        transport_type = transportation['type']
        duration = transportation['duration']
        plain_text_output.append(f"\n{i+1} - {name} ({transport_type}, {duration})")

    # Amenities
    amenities = ', '.join(hotel_data['amenities'])
    plain_text_output.append(f"\nAmenities: {amenities}")

    # Reviews breakdown
    plain_text_output.append("\nReviews Breakdown:")
    for review in hotel_data['reviews_breakdown']:
        plain_text_output.append(f" - {review['name']}: {review['positive']} positive, {review['negative']} negative, {review['neutral']} neutral mentions")

    return "\n".join(plain_text_output)

In [ ]:
Markdown(parse_hotel_data(bali_hotels["properties"][0]))

Hotel Name: The Ritz-Carlton, Bali 

Description: Zen-like quarters, some with butler service, in an upscale property offering refined dining & a spa.

Check-in Time: 3:00 PM

Check-out Time: 12:00 PM

Hotel Class: 5-star hotel

Overall Rating: 4.6 (based on 3614 reviews)

GPS coordinates: -8.830670999999999 , 115.21533099999999


Images:

https://d2hyz2bfif3cr8.cloudfront.net/imageRepo/7/0/151/470/91/rz-dpssw-private-pool-29237_Classic-Hor_O.jpg
https://d2hyz2bfif3cr8.cloudfront.net/imageRepo/7/0/147/874/299/dpssw-villa-0105-hor-clsc_O.jpg
https://d2hyz2bfif3cr8.cloudfront.net/imageRepo/7/0/147/874/243/dpssw-villa-0107-hor-clsc_O.jpg

 Rate per Night (before taxes): $287

Rate per Night (lowest): $347


Soruces of price :

1. The Ritz-Carlton, Bali


Nearby Places:

1 - I Gusti Ngurah Rai International Airport (Taxi, 29 min)

2 - Bejana, Indonesian Restaurant at The Ritz-Carlton, Bali (Walking, 1 min)

Amenities: Free Wi-Fi, Free parking, Pools, Hot tub, Air conditioning, Fitness centre, Spa, Beach access, Bar, Restaurant, Room service, Kitchen in some rooms, Airport shuttle, Full-service laundry, Accessible, Business centre, Child-friendly, Smoke-free property

Reviews Breakdown:
 - Property: 534 positive, 44 negative, 27 neutral mentions
 - Service: 507 positive, 74 negative, 18 neutral mentions
 - Nature: 212 positive, 29 negative, 15 neutral mentions

In [ ]:
bali_rental = {
  "search_metadata": {
    "id": "655335bab68f8b5cf2b314a3",
    "status": "Success",
    "json_endpoint": "https://serpapi.com/searches/7e9e85e70fe7f094/655335bab68f8b5cf2b314a3.json",
    "created_at": "2023-11-14 08:54:18 UTC",
    "processed_at": "2023-11-14 08:54:18 UTC",
    "google_hotels_url": "https://www.google.com/_/TravelFrontendUi/data/batchexecute?rpcids=AtySUc&source-path=/travel/search&hl=en&gl=us&rt=c&soc-app=162&soc-platform=1&soc-device=1",
    "raw_html_file": "https://serpapi.com/searches/7e9e85e70fe7f094/655335bab68f8b5cf2b314a3.html",
    "prettify_html_file": "https://serpapi.com/searches/7e9e85e70fe7f094/655335bab68f8b5cf2b314a3.prettify",
    "total_time_taken": 3.34
  },
  "search_parameters": {
    "engine": "google_hotels",
    "q": "Bali",
    "gl": "us",
    "hl": "en",
    "currency": "USD",
    "check_in_date": "2024-09-11",
    "check_out_date": "2024-09-12",
    "adults": 2,
    "children": 0,
    "vacation_rentals": true
  },
  "search_information": {
    "total_results": 14920
  },
  "properties": [
    {
      "type": "vacation rental",
      "name": "Yana Villas Kemenuh",
      "gps_coordinates": {
        "latitude": -8.555330276489258,
        "longitude": 115.28772735595703
      },
      "check_in_time": "2:00 PM",
      "check_out_time": "12:00 AM",
      "rate_per_night": {
        "lowest": "$40",
        "extracted_lowest": 40,
        "before_taxes_fees": "$33",
        "extracted_before_taxes_fees": 33
      },
      "total_rate": {
        "lowest": "$202",
        "extracted_lowest": 202,
        "before_taxes_fees": "$167",
        "extracted_before_taxes_fees": 167
      },
      "prices": [
        {
          "source": "Bluepillow.com",
          "logo": "https://www.gstatic.com/travel-hotels/branding/190ff319-d0fd-4c45-bfc8-bad6f5f395f2.png",
          "rate_per_night": {
            "lowest": "$40",
            "extracted_lowest": 40,
            "before_taxes_fees": "$33",
            "extracted_before_taxes_fees": 33
          }
        }
      ],
      "nearby_places": [
        {
          "name": "I Gusti Ngurah Rai International Airport",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "58 min"
            },
            {
              "type": "Public transport",
              "duration": "2 hr 22 min"
            }
          ]
        },
        {
          "name": "Warung Thepaon",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "10 min"
            }
          ]
        }
      ],
      "images": [
        {
          "thumbnail": "https://lh4.googleusercontent.com/proxy/1DpPI6Ipxyb3hCE5eBcnPWRiMvGGSgu-h50VkgCT_1ooT9mIPUIP2PvnfVHtHKpVCMJ6eG5x-lsYZRy4_cVg74cgY5qvDp-toQeT9bhkaSv15tUpFqVz8B0hr4JPdDsb6q-3Y_XJRhZmMuFVvLd0RU5Kg7GDqww=s287-w287-h192-n-k-no-v1",
          "original_image": "https://q-xx.bstatic.com/xdata/images/hotel/max1440x1080/223247563.jpg?k=17e4b19b4710d224007544d313b2e012e33475d53877bdea8cf036d8f6a60c1c&o="
        },
        {
          "thumbnail": "https://lh4.googleusercontent.com/proxy/HDK_ROG3ZEbuBkU5LEh2GIhGoiVm3Eof-1OBgO_50NdTLMh0BJ0W9r4scwbIi2bD53V897tTaTKYLP345Sf397KHtGijMdc2oFJuxdYGyygFwYECxX_88F8M38fjic99TnEJhqpXf1aNt_pV2uXDJD7K-ncqAA=s287-w287-h192-n-k-no-v1",
          "original_image": "https://q-xx.bstatic.com/xdata/images/hotel/max1440x1080/223247466.jpg?k=4bcc896b368c2bf0fc74edf39003f4ae18889e27d7c6df3d8ade34be887986be&o="
        },
        {
          "thumbnail": "https://lh6.googleusercontent.com/proxy/LRDW9oGz8oY9Pzis9Nli9yvOULgbnSqHd8tsg4mwHwLpBvCcm08r8F8xzFl9q5gymyiKlzBNNXdOqXPRR_mmBQndFePSlxDJcI7NSdSXR6wu8o3i-E7-nZiRnPJ2-BSmBpI8EDFMZjDBOkIBbRYcWfYP76b29g=s287-w287-h192-n-k-no-v1",
          "original_image": "https://s-light.tiket.photos/t/01E25EBZS3W0FY9GTG6C42E1SE/t_htl-dskt/tix-hotel/images-web/2020/10/31/9e51d564-cc30-4a2a-ab14-809d6deede87-1604146952201-3950d28982374ab237d1e13478487cd9.jpg"
        },
        ...
      ],
      "overall_rating": 3.994737,
      "reviews": 152,
      "location_rating": 3.3,
      "amenities": [
        "Air conditioning",
        "Airport shuttle",
        "Balcony",
        "Crib",
        "Fitness center",
        "Outdoor pool",
        "Smoke-free",
        "Washer",
        "Free parking",
        "Free Wi-Fi"
      ],
      "excluded_amenities": [
        "No beach access",
        "Not kid-friendly",
        "No elevator",
        "No fireplace",
        "No heating",
        "No hot tub",
        "No indoor pool",
        "No ironing board",
        "No kitchen",
        "No microwave",
        "No outdoor grill",
        "No oven stove",
        "No patio",
        "Not wheelchair accessible"
      ],
      "essential_info": [
        "Entire house",
        "Sleeps 2",
        "1 bedroom",
        "1 bathroom",
        "1 bed",
        "215 sq ft"
      ],
      "property_token": "ChcIyo2FjenOdOasxGgsvZy8xdGYyMTV2aBAB",
      "serpapi_property_details_link": "<SerpApi JSON endpoint>"
    },
    {
      "type": "vacation rental",
      "name": "Tirta sari",
      "gps_coordinates": {
        "latitude": -8.416190147399902,
        "longitude": 115.2825698852539
      },
      "check_in_time": "1:00 PM",
      "check_out_time": "6:00 AM",
      "rate_per_night": {
        "lowest": "$50",
        "extracted_lowest": 50,
        "before_taxes_fees": "$42",
        "extracted_before_taxes_fees": 42
      },
      "total_rate": {
        "lowest": "$252",
        "extracted_lowest": 252,
        "before_taxes_fees": "$208",
        "extracted_before_taxes_fees": 208
      },
      "prices": [
        {
          "source": "Bluepillow.com",
          "logo": "https://www.gstatic.com/travel-hotels/branding/190ff319-d0fd-4c45-bfc8-bad6f5f395f2.png",
          "rate_per_night": {
            "lowest": "$50",
            "extracted_lowest": 50,
            "before_taxes_fees": "$42",
            "extracted_before_taxes_fees": 42
          }
        }
      ],
      "nearby_places": [
        {
          "name": "I Gusti Ngurah Rai International Airport",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "1 hr 35 min"
            }
          ]
        },
        ...
      ],
      "images": [
        {
          "thumbnail": "https://lh6.googleusercontent.com/proxy/HfsdZpRpkAEHaei8gnPjCRbdwsd_EDHupUGPzYmEC5YrCZFxOGoZS5xaiBb7qlTO-d7GMlmdpBM3U1mXlBZknrNYL1IWGAsDhF7bWaB95A1AW35m0YQSBIBR5rXLYHfL8g-7ymvrV-bNa7FtW5ox1LMdZBdJtwI=s287-w287-h192-n-k-no-v1",
          "original_image": "https://s-light.tiket.photos/t/01E25EBZS3W0FY9GTG6C42E1SE/t_htl-dskt/tix-hotel/images-web/2023/05/17/b14d0a14-388c-4eff-8cfa-71ba9665045c-1684315756338-947f4308781850dc6b18828071a0c6f3.jpg"
        },
        ...
      ],
      "overall_rating": 4.225,
      "reviews": 12,
      "location_rating": 3.6,
      "amenities": [
        "Air conditioning",
        "Balcony",
        "Crib"
      ],
      "excluded_amenities": [
        "No airport shuttle",
        "No beach access",
        "Not kid-friendly",
        ...
      ],
      "essential_info": [
        "Entire villa",
        "Sleeps 3",
        "1 bedroom",
        "1 bathroom",
        "269 sq ft"
      ],
      "property_token": "ChcIyo2FjenOuZ8xGgsvZy8xdGds4TV2aBAB",
      "serpapi_property_details_link": "<SerpApi JSON endpoint>"
    },
    {
      "type": "vacation rental",
      "name": "Keramas Sacred River Village",
      "link": "https://deals.vio.com?sig=73aca13c7f952d2641c156f3e69125e1eb497c325f122828ee5aa8797168b9a12d32303331333438363233&turl=https%3A%2F%2Fwww.vio.com%2FHotel%2FSearch%3FhotelId%3D5255198%26utm_source%3Dgha-vr%26utm_campaign%3Dstatic%26openHotelDetails%3D1",
      "gps_coordinates": {
        "latitude": -8.586440086364746,
        "longitude": 115.3296127319336
      },
      "check_in_time": "2:00 PM",
      "check_out_time": "12:30 AM",
      "rate_per_night": {
        "lowest": "$46",
        "extracted_lowest": 46,
        "before_taxes_fees": "$38",
        "extracted_before_taxes_fees": 38
      },
      "total_rate": {
        "lowest": "$228",
        "extracted_lowest": 228,
        "before_taxes_fees": "$188",
        "extracted_before_taxes_fees": 188
      },
      "prices": [
        {
          "source": "Vio.com",
          "logo": "https://www.gstatic.com/travel-hotels/branding/7287187d-2586-494f-92ff-726979e94c2a.png",
          "rate_per_night": {
            "lowest": "$46",
            "extracted_lowest": 46,
            "before_taxes_fees": "$38",
            "extracted_before_taxes_fees": 38
          }
        }
      ],
      "nearby_places": [
        {
          "name": "I Gusti Ngurah Rai International Airport",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "52 min"
            }
          ]
        },
        {
          "name": "Warong Legong",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "4 min"
            }
          ]
        }
      ],
      "images": [
        {
          "thumbnail": "https://lh5.googleusercontent.com/proxy/yXB-H-lTQ1MH1bWIMXAU3JRRJCiLC6apOl-dkE2FhpTQLljrYhEayGK0_a5hGaLnJlVpKh072yvLFV2mH-brVIpRGBBftAvu-d3bAbjHK9EKMmjzoARdCsUW_KnuvtIZwvC0r6DAyevg1JL2dDflEB2PBoV8qFE=s287-w287-h192-n-k-no-v1",
          "original_image": "https://p.fih.io/v2/nSr0o-3gbZN8o1xMecXCcG-BkDeQVo3k-wOeVogkTe7ajuFOX7O-WA8aRWhXz1jsToZif_TsA_xCWfhWRa0Z3Ke1vT1z90khCU2Da7x0-xiShVn8GuC9vj7KBaOhg-vgKl4c_rzAKF78OTF4en3ketkYyx1j1dLbUfhlOgkNcIcn9PwNtk_AU2csbUk6dd9sGqAEVmr2qjQMPI095Pdv3pEtFPIxOxxOPC-YWoFEg4yXvCZQ3nG9m-EOZqnW2lodL_hO30Ezy-jlqKVHa0uB_8O59smfYWZisPxpsuKafAZkFxNkAjW1f-Y-DpE9Rgh64AZ_KQd1YCIQC1WVxAG4bqbEaIBWIgb2"
        },
        {
          "thumbnail": "https://lh3.googleusercontent.com/proxy/K3K-ywMEqxZwT8DWkI4v8_qukJDwY01i8mOZDP6ppAlbh9GQcGotrQDWO0NSBo3GW5x_hXnxGGZpvBVCrsNtSfk10DEfSB4qYaybt5TLWvxjwPXrUQCEGqkKEHDdcTpSxGg8MvgmMe-GBrQLO0MjWLxn4XTO7LU=s287-w287-h192-n-k-no-v1",
          "original_image": "https://p.fih.io/v2/nSr0o-3gbZN8o1xMecXCcG-BkDeQVo3k-wOeVogkTe7ajuFOX7O-WA8aRWhXz1jsToZif_TsA_xCWfhWRa0ZrqOuunYD91xgFQ_ea84R1z7jkWbdLumZpU_AM4PivpuFGD9eyu67MlTVFzF4en3ketkYyx1j1dLbUfhlOgkNcIcn9PwNtk_AU2csbUk6dd9sGqAEVmr2qjQMPI095Pdv3pEtFPIxOxxOPC-YWoFEg4yXvCZQ3nG9m-EOZqnW2lodL_hO30Ezy-jlqKVHa0uB_8O59smfYWZisPxpsuKafAZiAxNnXzS1fLw-U5E9Rgh64AZ_KQd1YCKpLdhy5WGXi-CaBHchStHA"
        },
        {
          "thumbnail": "https://lh4.googleusercontent.com/proxy/Rm6W1Ti3m6XIgyV8UE786I_f4EPBIFcFesq8eJbgqc0sgv1G6bBwfzLK8FYO6jmW0idKZsfkL0Ozo43Fr8J0AVaCEAlh71GTZwfBgafPG-craBwN__kAGzd9KukM_HLyxLx1Qf9SQ5NvW9R7P5GwK4fkB8mowuI=s287-w287-h192-n-k-no-v1",
          "original_image": "https://p.fih.io/v2/nSr0o-3gbZN8o1xMecXCcG-BkDeQVo3k-wOeVogkTe7ajuFOX7O-WA8aRWhXz1jsToZif_TsA_xCWfhWRa0ZvcfmhAQK8GZjLRmGdIFV8DvNriDeH-2U-Dz7cr-i_Z-2KgB83r7cGl_FJTF4en3ketkYyx1j1dLbUfhlOgkNcIcn9PwNtk_AU2csbUk6dd9sGqAEVmr2qjQMPI095Pdv3pEtFPIxOxxOPC-YWoFEg4yXvCZQ3nG9m-EOZqnW2lodL_hO30Ezy-jlqKVHa0uB_8O59smfYWZisPxpsuKafAZkFy1rXyCmePQpaZE9Rgh64AZ_KQd1YCJpER13G6kZmqrt3sn5ocKZ"
        },
        ...
      ],
      "overall_rating": 4.35,
      "reviews": 152,
      "location_rating": 2.8,
      "amenities": [
        "Air conditioning",
        "Airport shuttle",
        "Outdoor pool",
        ...
      ],
      "excluded_amenities": [
        "No beach access",
        "No elevator",
        "No fireplace",
        ...
      ],
      "essential_info": [
        "Sleeps 10",
        "8 bedrooms",
        "5 bathrooms"
      ],
      "property_token": "ChcIyo2Fjdjs2uZ8xGgsvZy8xdGYyMTV2aBAB",
      "serpapi_property_details_link": "<SerpApi JSON endpoint>"
    },
    ...
  ],
  "serpapi_pagination": {
    "current_from": 1,
    "current_to": 18,
    "next_page_token": "CBI=",
    "next": "https://serpapi.com/search.json?adults=2&check_in_date=2024-09-11&check_out_date=2024-09-12&children=0&currency=USD&engine=google_hotels&gl=us&hl=en&next_page_token=CBI%3D&q=Bali&vacation_rentals=true"
  }
}

In [ ]:
def get_food_info(
    food_name : str,
    location : str,
    dining_option: str
):
    """
    Fetch food details from the Google Food API based on location and dining options.

    Args:
        food_name (str):
            The name of the food or cuisine to search for. This can be a specific dish or a general type of food.
            Example: "pizza", "sushi"

        location (str):
            The location to search for food options. coordinates (latitude, longitude).
            Example: "San Francisco", "37.7749,-122.4194"

        dining_option (str):
            The type of dining option desired. Must be one of the following: "pickup" or "delivery".
            Example: "pickup"

    Returns:
        dict: A dictionary containing details about available food options based on the specified criteria.
    """

    pass

In [ ]:
Satay_food = {
   "search_metadata":{
      "id":"66ea7d50b9458db62bdeebe6",
      "status":"Success",
      "json_endpoint":"https://serpapi.com/searches/657afaf121dd723f/66ea7d50b9458db62bdeebe6.json",
      "created_at":"2024-09-18 07:12:16 UTC",
      "processed_at":"2024-09-18 07:12:16 UTC",
      "google_food_url":"https://food.google.com/food/homepage?q=Satay&hl=id&lat=-8.3405389&lng=115.0919509",
      "raw_html_file":"https://serpapi.com/searches/657afaf121dd723f/66ea7d50b9458db62bdeebe6.html",
      "total_time_taken":1.29
   },
   "search_parameters":{
      "engine":"google_food",
      "q":"Satay",
      "hl":"id",
      "gl":"id",
      "lat":"-8.3405389",
      "lng":"115.0919509",
      "location_requested":"Bali, Indonesia",
      "location_used":"Bali,Indonesia"
   },
   "search_information":{
      "local_results_state":"Results for exact spelling"
   },
   "local_results":[
      {
         "title":"Gourmet Sate House",
         "rating":4.4,
         "reviews_original":"(841)",
         "reviews":841,
         "distance":"26.0 mi",
         "address":"Jl. Dewi Sri No.101",
         "hours":"Open ⋅ Closes 11 PM (WITA)",
         "delivery_time": "Delivery in 30 min",
         "delivery_fee": "$3.99 delivery fee",
         "restaurant_id":"/g/11b74l74k3",
         "images":[
            "https://lh3.googleusercontent.com/gps/AA0Xun1GXVXNF2EnBC3nPKvfz_8PbYt4zZd_7tZZ_OAPKxRa-J4yQfgZsvoYX5N2Z0rOsZqiNxbTIkPFNyRbNr2ccBTvKA=w1333-h1000-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun3yjoRDFqONvAP_Bdxajv6zPxT2BxM2qn2jKGY_wUQ03X8qGhUVyxfVg4SvtsDZlYy9FEZ0RvhmYPDHxc7vr2ZuPg=w1000-h1777-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun0l-DHxTytNwKdblgxQ2lPZJx-p4aMN9Zpm4-ITAtykMnh2n9UB-pO3_4gt1O90Hftu4EFckIotXt8aLzCz3zmtiw=w1333-h1000-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun3cWYSlJUZHNai6m1txsq-pAaasOsjWHOMAXuMtQcE3h6fJtWpm2-m5eWjdWyh4aATFjWben8fmSWUxg1TeOCtlXA=w1333-h1000-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun2aSQSXqtosfcQUZmgl5QSjzGejf39uLzxh0NMkET4T4vQ0rEvX1Z2CbLfQ6HtsN5aW6bF2N17YGWfgOKXjYl2unA=w1331-h1000-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun1obvGKzsg-5ji1kOi9amNylWscDd9ium6bQL0yxBZStUFEPJRS4bp6UUAtmlc_sL3FOWJQ6JP0FoahLfLSm3iQnw=w1333-h1000-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun1qDJEm4adoRjsmvHpL_KMavhZtppuFHwTKIwGHPNDBlhqlf3Bj6HYhVCt986tuaO3Jk6k2jXT5JKAGVnoaNfHhjA=w1333-h1000-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun0jtZAg2Ba5t_zTxSG60HpeFZ-tBLumFmpxM-3Y8O5VxF2B3gVkL4mZBWWOHQ9nCY8Nh97z0I0pmoScq0UI90_UjQ=w1333-h1000-k-no"
         ],
         "links":{
            "order":"https://food.google.com./chooseprovider?restaurantId=/g/11b74l74k3&orderType=2&sa=X&ved=2ahUKEwjIkre1-MuIAxXgFJ0JHSoaPd0Q-7AKegQIABAn",
            "phone":"+62 821-4410-1909",
            "directions":"https://www.google.com/maps/dir//Gourmet+Sate+House/data=!4m6!4m5!1m1!4e2!1m2!1m1!1s0x2dd246cf415ee4ed:0x17acce2a0056e210?sa=X&ved=2ahUKEwjIkre1-MuIAxXgFJ0JHSoaPd0Q9LIJegQIABAv"
         }
      },
      {
         "title":"SATAY GOnDrong Bali",
         "rating":4.5,
         "reviews_original":"(19)",
         "reviews":19,
         "distance":"20.7 mi",
         "address":"Jl. Raya Padonan Banjar kulibul kangin No.98",
         "hours":"Open ⋅ Closes 10 PM (WITA)",
         "restaurant_id":"/g/11hkw4jwv5",
         "images":[
            "https://lh3.googleusercontent.com/gps/AA0Xun148Qr-gD7ftjozaZDxgCjwTpnoIqUDlqzGTCjIFdFTWVZ1iP1TbNfBzFcKS6tPiXUwcqg0dJgvOWGU3kvbI1P8bw=w1333-h1000-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun23N4ePHYiUPX5fd3XlrJCmucwPfRWfo6z75AKXEcxuLB2V-2iuHw3HFjlILPSWhpcY2ufYON369VRmriGmtd_Ltg=w1000-h1000-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun3ZOzQWgnc8rDdlThTVNC-EyamwhVRdAuztxLfgnQIW_cH5QYs_LnypibbFsOw7ttL9-z_wyl4JVRSD-I9ektdHsg=w1000-h1000-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun1lIbEcWVwgmJ6-XqlvEXtM_Sx2QTFugOKtXGWUmS42bmVoXT889PbbKb0tGW5XW0dgwzDVX2TS82mVzlsNCKlwIQ=w1000-h1333-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun35rOyo9dwL-hpEhSaVA-nHY4MlD1scs9OxQ8MuahS8qCAxzGXMP4V12MsMUUMjZLj1nasfLrjuNXCvOM57-8TmDQ=w1000-h1333-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun3eArBKToNOB5QLmIMLqlZ1Ytqxmu4yCgKsecTElHFZPNyRtjXBOT0mEbsnXOOXfTYaDzEyWwxbl8B3eA1wrKSf5w=w1000-h1000-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun07tPhRrgD0V1lKdGVQTZ7ip8Gyrty6If9IJ5t1GtZL9zjeOhwwh4P9MLooG4lz1NhtKKAMikfd2Cx-IzhsK_73sg=w1000-h1333-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun18MjDRN6ZpPzm6KNPolAwuxEN2U6skJPcbK2JtJ-dyzq70laQZt2larr-GTpOHJQgAtZI0P3z9_K7mth2Oep0BVw=w1333-h1000-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun3IOzQ00AAly8hz_ID4adFLD2Kcrn1WruSJlEbNrbHFA0VmuB9x_8V4lY4dtOc53m6uxypFPKz8F8VU8B9heSFOCg=w1000-h1000-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun2eaBL8EMofg89-gVBBnc5CISUaUhxdWNYNiKZScj62lZIws7AkYhyVP7hFy9Jp6YAT6MEZETKTTORWEgM_wY5a-A=w1000-h1000-k-no"
         ],
         "links":{
            "order":"https://food.google.com./chooseprovider?restaurantId=/g/11hkw4jwv5&orderType=2&sa=X&ved=2ahUKEwjIkre1-MuIAxXgFJ0JHSoaPd0Q-7AKegQIABAx",
            "phone":"+62 812-2879-2367",
            "directions":"https://www.google.com/maps/dir//SATAY+GOnDrong+Bali/data=!4m6!4m5!1m1!4e2!1m2!1m1!1s0x2dd24704c67bee1d:0xd9c3578a1eb2b5a5?sa=X&ved=2ahUKEwjIkre1-MuIAxXgFJ0JHSoaPd0Q9LIJegQIABA5"
         }
      },
      {
         "title":"D'SATAY",
         "rating":4.6,
         "reviews_original":"(13)",
         "reviews":13,
         "distance":"19.2 mi",
         "address":"94MF+JCH",
         "hours":"Open ⋅ Closes 11 PM (WITA)",
         "restaurant_id":"/g/11qykbnqdn",
         "images":[
            "https://lh3.googleusercontent.com/gps/AA0Xun1tmnxFOXrBfJcr5alXCW97uF8Kt6msR7ih7LxcYkxI7liEkBEC5qqFZ2pDMBk2ammJyMfoYRSa7-KrSnRWfirT3w=w1000-h1333-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun3p_2UxBftM3Bwyq9ATLfUI1y2Ilj3VX6KTeXgzb41B41SQFC66YpEiQTLNejh8NrkKZ_IXu_Pjj7L5MVPc01X02w=w1000-h1333-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun16jEfK-27wtw8yqJo1Szcdx_y6VTx7EY5UedguGS4azX4PWSnY_L7I8vQgnYAcgrdzwBc4T2sY4_V85I-pUoWmhA=w1000-h1333-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun1T4m4YmHnvW5jcZl9ETQfnqy3DARF15Yj5W-0IF0ElA2OijUO7uaOKJf4Q8O9LsQ7BCU4ZZHNNHnV6IEzgevPNuQ=w1000-h1333-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun04KyDQedLIMJagZq1KTNP3HMnrKUN5GPBFYj2IHick6F4hZ3uTrJJzm-_I0BxXPE3WC_jrFTPHihw2Lt0RYu0HQA=w2109-h1000-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun2Al1EKUIv8hn5dbjH0f9rO568CNE1fcHfSqH6MFSVVXrMistydUcrGzxrh32YuG7JEBalsTZj00E5r_o4pYWb5Sg=w1333-h1000-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun1WJn0rcIx1d8opjrIc3YzPCP8EGj-DjStKNSXuSnpjHyP1SvzpVdhd-81xAocZaDVyIXesDEMGpcLPIaV-zBfrkg=w1000-h1777-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun0D0ag-CTbwCp6yORGa3K5AJtH0fqyQGFALnjPMnFsF_1j4CEoyxPXNeTwLCw_06ye6OsFse-QImjGkMTGZaDWaIg=w1000-h1777-k-no"
         ],
         "links":{
            "order":"https://food.google.com./chooseprovider?restaurantId=/g/11qykbnqdn&orderType=2&sa=X&ved=2ahUKEwjIkre1-MuIAxXgFJ0JHSoaPd0Q-7AKegQIABA7",
            "phone":"+62 821-4433-1095",
            "directions":"https://www.google.com/maps/dir//D%27SATAY/data=!4m6!4m5!1m1!4e2!1m2!1m1!1s0x2dd239b7671878db:0x47a68403dcaf2ca2?sa=X&ved=2ahUKEwjIkre1-MuIAxXgFJ0JHSoaPd0Q9LIJegQIABBD",
            "website":"https://gofood.link/a/zYRUSU5?sa=X&ved=2ahUKEwjIkre1-MuIAxXgFJ0JHSoaPd0Q-IkKegQIABBE"
         }
      },
      {
         "title":"Warung Sate Madura Pak Ayex",
         "rating":4.5,
         "reviews_original":"(230)",
         "reviews":230,
         "distance":"23.5 mi",
         "address":"No., Jalan Tangkuban Perahu No.36CX, Kerobokan Kelod, Kuta Utara",
         "hours":"Open ⋅ Closes 10 PM (WITA)",
         "restaurant_id":"/g/11bw4w0cl4",
         "images":[
            "https://lh3.googleusercontent.com/gps/AA0Xun2j-sCEw6g0If2tkuBw2AL3ppRVopXVWI-Tn1He-pjazYbtd_WyBFsyj2V5KOJWf04K61t3Un5X6oEy7_z0WI8zWg=w1333-h1000-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun3gsRHMgNGM_CmjpTJT63Rdz7YJmiAjU2xw5DYL44YA13X6tNy9OZt8piBCmg1NTYdosjc13IckdT5jE5ySzsPQKg=w1000-h1777-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun3EzXXMyv7sTNfUfXrCMXMOkBfp-pwef5M5MixrUNOCTXd1snLUpXXsimkpOyQcpyCfvKE-hlfLcEPuQ5VAoE4kDw=w1000-h1162-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun0IZby6YDSvi6hh6r53jZUs5WFsP7XTBJQZYuRb1NdVgZxyXjPn2d8k8RB2ydl9ZukIj4Y-AGN1skk42D3H-e-GJw=w1331-h1000-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun0DBeMRsR1KMMeTHs5qtVNKtgNfmUc6HVDsfn1BUMbWp7PHxhFf2uSLh57eKfiJhdZew0IXznR_aJnHUkLYPc0omQ=w1333-h1000-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun2UXDujhncVA_kTaR4UAhXYS2CryYIXNUy5nsJRnjenJ7UKsiS3SqrmcfFBlU7RejQSX5XpRYqY8nbPxkmdwF1IAA=w1000-h1336-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun0DHW2miw5YVjP2RVn3Y5rPfD3TxDZ5zdNIDiSuvW_UbpITdBtITcKbZYIXevZ3FoQJM-F0ahf4nzPH3PjuP46HtQ=w1000-h1333-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun0LYwsuLg8FgBT0O99r-_hmWM7wQPWu3zJbX8KAMko9jMtBxzhGwks7Yegde7tp9wLXLX73FF_GjAT9bAabk3ehBw=w1333-h1000-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun2EdIbZsyvwdWEQal7i26U3ZlhciCMCvRh8WByXcfZOySCj7ijFLRG6nX6m0An08aBk1LeXmdjujcQ-eLMqyKHSKg=w1000-h1333-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun3APOo2ze8TJFTXnABNQGSOLyTOPcsuhJgp9hPrLnPX4nF_iK5ZxkQnZ6Wi2yuZ5MHy9uhcehqqL2uaHN5n4rxh5w=w1000-h1323-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun2H0Hj5yXKiyzriMpWy4BLhw_1ytIzq29T7aWLddVgabPy_cy7P-RzY1IY9QaDxE-DX2XEeqnPnp_7nmvnyhaKxew=w1000-h1333-k-no"
         ],
         "links":{
            "order":"https://food.google.com./chooseprovider?restaurantId=/g/11bw4w0cl4&orderType=2&sa=X&ved=2ahUKEwjIkre1-MuIAxXgFJ0JHSoaPd0Q-7AKegQIABBG",
            "phone":"+62 819-9961-5357",
            "directions":"https://www.google.com/maps/dir//Warung+Sate+Madura+Pak+Ayex/data=!4m6!4m5!1m1!4e2!1m2!1m1!1s0x2dd24740f32bb1fd:0xd21281768bb65194?sa=X&ved=2ahUKEwjIkre1-MuIAxXgFJ0JHSoaPd0Q9LIJegQIABBO"
         }
      },
      {
         "title":"Sate Babi Tut Narutto",
         "rating":4.6,
         "reviews_original":"(254)",
         "reviews":254,
         "distance":"17.5 mi",
         "address":"Jl. Tambaksari No.27",
         "hours":"Open ⋅ Closes 6 PM (WITA)",
         "restaurant_id":"/g/11c5sb0sdm",
         "images":[
            "https://lh3.googleusercontent.com/gps/AA0Xun23Ezl6ieksd67hwyDzCmmanrNwkvb1GBKPHWmIelCq4s3F2qYKhIilfXyqB31awMH0mxJHR-zwZZme7bgSWELc-g=w1333-h1000-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun1tWJfDQPwj7B9xbuvzmIiURkTEM95B0G39ASiF0PL8jDYsJPk1471Mw2_GEtTWEkz4z7W7WJUi3MhzWA9Z5f_5lA=w1000-h1777-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun3x7Om7i17K43PwW1zfSMVPdqUE4j6T6qbBtH5tPEDuapv9pZiSiBb2NjgBHHyujbrX8AQFUpXuWp4yTlE5RZyxbA=w1333-h1000-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun2Zq904EgTdb58ebYnmBovFtReI41mTySywzD1uxSr4GEyJXl4xhfOTSvZS-7WS7dFzKiuchVRC_usG2s6OjzP0Zg=w1000-h1000-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun1k_qz_Nfrv-2hfdLq5bvLP0ca3aWbvGtYW7N9AE02sCi1WyEp2BwwNly7EFNCci8PxSc35vYG7VFGl0Z08lpq1bw=w1000-h1333-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun1qiYWJvYb-xL7v3IxnJldSaCT7NWxBfJC9oa74QdT0eNaKv0k6ypruSZqrH7lQwm3EVhYRmAzK7nHsoDFzkAbNyQ=w1000-h1333-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun2r8KwoChCL_518jQWF8T_juDCN1o9euyryUo2Vq72ej8nqEeX_G5zGIrbWCSNGKAHYTAaaYbCWCUCNExG81lxYOw=w1000-h1333-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun0WTOYcWTmAyRfZ444up_OoeTC-F7CbF9PQm8eQW6blX_C9sEFK5ZRt6heiLcFA8o-DlD0s3luGE7Uw7NpueJ8yDA=w1783-h1000-k-no"
         ],
         "links":{
            "order":"https://food.google.com./chooseprovider?restaurantId=/g/11c5sb0sdm&orderType=2&sa=X&ved=2ahUKEwjIkre1-MuIAxXgFJ0JHSoaPd0Q-7AKegQIABBQ",
            "phone":"+62 878-6199-7720",
            "directions":"https://www.google.com/maps/dir//Sate+Babi+Tut+Narutto/data=!4m6!4m5!1m1!4e2!1m2!1m1!1s0x2dd2395aec850de1:0xe9f350581e456e65?sa=X&ved=2ahUKEwjIkre1-MuIAxXgFJ0JHSoaPd0Q9LIJegQIABBY"
         }
      },
      {
         "title":"Sate Khas Senayan Kuta Beachwalk",
         "rating":4.2,
         "reviews_original":"(254)",
         "reviews":254,
         "type":"Indonesian",
         "distance":"26.4 mi",
         "address":"1-E22, Beachwalk Shopping Center, Lantai, Jl. Pantai Kuta",
         "hours":"Open ⋅ Closes 10 PM (WITA)",
         "restaurant_id":"/g/11rr563t34",
         "images":[
            "https://lh3.googleusercontent.com/gps/AA0Xun1U6Fqr_ZAl5AKJUDEJve99cCqyWpN7yRzEut5z82XeU4lz7rjzm5O8JDupO2TIvpv1oMZITE2z3eUMRDhX7sEXjA=w1000-h1000-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun3Kq4etK-vdSnuG5et_UxvCSPYcQvcCnl_2jcIezDhf4AUP8erGNlwkDwQ3KyZP9VUv1hjz-Czx4osXPxx_DwUfWw=w1000-h1331-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun0bjmi8_y2U8aW1Dwo-AXWB9I9KFZBc9ySYhkdHdyYHuLQfGPfUdv5Q-sILNDcEp13wEUuaShjmWFlsFVjPf4nMlA=w1000-h1333-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun2VSEl1xG0psrwLHOJmbx3seetoNPszptDr8Bo0XkSPiKESDtcAagmGySjmRRBDUEgRaY0KRFltfB0t5nV07BIPRg=w1000-h1777-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun3siK_D-dCFd-Syl7tmXkImUQkDYrmEQVgNzuCPRNRDv_sH-0TKo5bNU4AswQlH2W9vytLfSQFjnsyaCpZR9lSAqQ=w1000-h1333-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun1m3_lpxF12Eqn9BSpuDbDTjHaNCh5FKwN_jR2cfeRYbyoEYlrIT-jScoy2_H1jvY1cNDNy5i1GIvT1fNE4pdgnJQ=w1000-h1249-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun2yb0fPtX9lwg1mV3vsdLFrrcpXFZLMnB0p1ZvLq0_ici2yiUd2WXWFhBD0ZY6JE6xJQ5V8UHVQO6n7UHFDqsg9nw=w1000-h1000-k-no",
            "https://lh3.googleusercontent.com/gps/AA0Xun0jEgKpbUeSNQIz-klmSpQjUYAENKSD2HS5txL0e3DzinFTZEeZ-uqPuaXjpFxtmmgXC6UHMLqUoRuYTod9xM24gQ=w1000-h1333-k-no"
         ],
         "links":{
            "order":"https://food.google.com./chooseprovider?restaurantId=/g/11rr563t34&orderType=2&sa=X&ved=2ahUKEwjIkre1-MuIAxXgFJ0JHSoaPd0Q-7AKegQIABBa",
            "phone":"+62 361 4757101",
            "directions":"https://www.google.com/maps/dir//Sate+Khas+Senayan+Kuta+Beachwalk/data=!4m6!4m5!1m1!4e2!1m2!1m1!1s0x2dd24718b3df548b:0xbda31d89d5aa96a5?sa=X&ved=2ahUKEwjIkre1-MuIAxXgFJ0JHSoaPd0Q9LIJegQIABBi",
            "website":"https://sks.sarirasa.co.id/?sa=X&ved=2ahUKEwjIkre1-MuIAxXgFJ0JHSoaPd0Q-IkKegQIABBj"
         }
      }
   ],
   "serpapi_pagination":{
      "next_page_token":"W1tbImdXdk0xZCIsIltbMV0sXCJTYXRheVwiLFtbWzEsW251bGwsWzJdXV1dXSwxOCxcIkNCST1cIixudWxsLDEsXCJJQUU9XCIsW251bGwsbnVsbCxudWxsLFstOC4zNDA1Mzg5LDExNS4wOTE5NTA5XV0sbnVsbCwxXSIsbnVsbCwiZ2VuZXJpYyJdXV0=",
      "next":"https://serpapi.com/search.json?engine=google_food&gl=id&hl=id&lat=-8.3405389&lng=115.0919509&location=Bali%2C+Indonesia&next_page_token=W1tbImdXdk0xZCIsIltbMV0sXCJTYXRheVwiLFtbWzEsW251bGwsWzJdXV1dXSwxOCxcIkNCST1cIixudWxsLDEsXCJJQUU9XCIsW251bGwsbnVsbCxudWxsLFstOC4zNDA1Mzg5LDExNS4wOTE5NTA5XV0sbnVsbCwxXSIsbnVsbCwiZ2VuZXJpYyJdXV0%3D&q=Satay"
   }
}

In [ ]:
Satay_food["local_results"][0]

{'title': 'Gourmet Sate House',
 'rating': 4.4,
 'reviews_original': '(841)',
 'reviews': 841,
 'distance': '26.0 mi',
 'address': 'Jl. Dewi Sri No.101',
 'hours': 'Open ⋅ Closes 11 PM (WITA)',
 'delivery_time': 'Delivery in 30 min',
 'delivery_fee': '$3.99 delivery fee',
 'restaurant_id': '/g/11b74l74k3',
 'images': ['https://lh3.googleusercontent.com/gps/AA0Xun1GXVXNF2EnBC3nPKvfz_8PbYt4zZd_7tZZ_OAPKxRa-J4yQfgZsvoYX5N2Z0rOsZqiNxbTIkPFNyRbNr2ccBTvKA=w1333-h1000-k-no',
  'https://lh3.googleusercontent.com/gps/AA0Xun3yjoRDFqONvAP_Bdxajv6zPxT2BxM2qn2jKGY_wUQ03X8qGhUVyxfVg4SvtsDZlYy9FEZ0RvhmYPDHxc7vr2ZuPg=w1000-h1777-k-no',
  'https://lh3.googleusercontent.com/gps/AA0Xun0l-DHxTytNwKdblgxQ2lPZJx-p4aMN9Zpm4-ITAtykMnh2n9UB-pO3_4gt1O90Hftu4EFckIotXt8aLzCz3zmtiw=w1333-h1000-k-no',
  'https://lh3.googleusercontent.com/gps/AA0Xun3cWYSlJUZHNai6m1txsq-pAaasOsjWHOMAXuMtQcE3h6fJtWpm2-m5eWjdWyh4aATFjWben8fmSWUxg1TeOCtlXA=w1333-h1000-k-no',
  'https://lh3.googleusercontent.com/gps/AA0Xun2aSQSXqtosfcQUZ

In [ ]:
def parse_food_info(response: dict):
    """
    Parse the food details from the JSON response into a plain text format.

    Args:
        response (dict): A dictionary containing the food details from the API response.

    Returns:
        str: A plain text formatted string containing the parsed food details.
    """
    title = response.get("title", "Unknown Title")
    rating = response.get("rating", "No rating")
    reviews = response.get("reviews", "No reviews")
    distance = response.get("distance", "Unknown distance")
    address = response.get("address", "Unknown address")
    hours = response.get("hours", "No hours available")
    delivery_time = response.get("delivery_time", "Delivery time not available")
    delivery_fee = response.get("delivery_fee", "Delivery fee not available")
    restaurant_id = response.get("restaurant_id", "No restaurant ID")
    images = response.get("images", [])
    order_link = response.get("links", {}).get("order", "No order link available")
    phone = response.get("links", {}).get("phone", "No phone number available")
    directions_link = response.get("links", {}).get("directions", "No directions link available")

    # Create a plain text format
    plain_text = f"""
    Restaurant: {title} \n
    Rating: {rating} ({reviews} reviews) \n
    Distance: {distance}\n
    Address: {address}\n
    Operating Hours: {hours}\n

    Delivery Details:
    Delivery Time: {delivery_time}
    Delivery Fee: {delivery_fee}

    Restaurant ID: {restaurant_id}

    Contact:
    Phone: {phone}
    Order Link: {order_link}
    Directions: {directions_link}

    Image Links:
    """

    # Adding all the images
    if images:
        for idx, img in enumerate(images, 1):
            plain_text += f"    {idx}. {img}\n"
    else:
        plain_text += "    No images available.\n"

    return plain_text.strip()


# parsed_response_plain_text = parse_food_info_to_plain_text(response_with_missing_fields)
# print(parsed_response_plain_text)


In [ ]:
Markdown(parse_food_info(Satay_food["local_results"][0]))

Restaurant: Gourmet Sate House 

    Rating: 4.4 (841 reviews) 

    Distance: 26.0 mi

    Address: Jl. Dewi Sri No.101

    Operating Hours: Open ⋅ Closes 11 PM (WITA)


    Delivery Details: 
    Delivery Time: Delivery in 30 min 
    Delivery Fee: $3.99 delivery fee

    Restaurant ID: /g/11b74l74k3

    Contact:
    Phone: +62 821-4410-1909
    Order Link: https://food.google.com./chooseprovider?restaurantId=/g/11b74l74k3&orderType=2&sa=X&ved=2ahUKEwjIkre1-MuIAxXgFJ0JHSoaPd0Q-7AKegQIABAn
    Directions: https://www.google.com/maps/dir//Gourmet+Sate+House/data=!4m6!4m5!1m1!4e2!1m2!1m1!1s0x2dd246cf415ee4ed:0x17acce2a0056e210?sa=X&ved=2ahUKEwjIkre1-MuIAxXgFJ0JHSoaPd0Q9LIJegQIABAv

    Image Links:
        1. https://lh3.googleusercontent.com/gps/AA0Xun1GXVXNF2EnBC3nPKvfz_8PbYt4zZd_7tZZ_OAPKxRa-J4yQfgZsvoYX5N2Z0rOsZqiNxbTIkPFNyRbNr2ccBTvKA=w1333-h1000-k-no
    2. https://lh3.googleusercontent.com/gps/AA0Xun3yjoRDFqONvAP_Bdxajv6zPxT2BxM2qn2jKGY_wUQ03X8qGhUVyxfVg4SvtsDZlYy9FEZ0RvhmYPDHxc7vr2ZuPg=w1000-h1777-k-no
    3. https://lh3.googleusercontent.com/gps/AA0Xun0l-DHxTytNwKdblgxQ2lPZJx-p4aMN9Zpm4-ITAtykMnh2n9UB-pO3_4gt1O90Hftu4EFckIotXt8aLzCz3zmtiw=w1333-h1000-k-no
    4. https://lh3.googleusercontent.com/gps/AA0Xun3cWYSlJUZHNai6m1txsq-pAaasOsjWHOMAXuMtQcE3h6fJtWpm2-m5eWjdWyh4aATFjWben8fmSWUxg1TeOCtlXA=w1333-h1000-k-no
    5. https://lh3.googleusercontent.com/gps/AA0Xun2aSQSXqtosfcQUZmgl5QSjzGejf39uLzxh0NMkET4T4vQ0rEvX1Z2CbLfQ6HtsN5aW6bF2N17YGWfgOKXjYl2unA=w1331-h1000-k-no
    6. https://lh3.googleusercontent.com/gps/AA0Xun1obvGKzsg-5ji1kOi9amNylWscDd9ium6bQL0yxBZStUFEPJRS4bp6UUAtmlc_sL3FOWJQ6JP0FoahLfLSm3iQnw=w1333-h1000-k-no
    7. https://lh3.googleusercontent.com/gps/AA0Xun1qDJEm4adoRjsmvHpL_KMavhZtppuFHwTKIwGHPNDBlhqlf3Bj6HYhVCt986tuaO3Jk6k2jXT5JKAGVnoaNfHhjA=w1333-h1000-k-no
    8. https://lh3.googleusercontent.com/gps/AA0Xun0jtZAg2Ba5t_zTxSG60HpeFZ-tBLumFmpxM-3Y8O5VxF2B3gVkL4mZBWWOHQ9nCY8Nh97z0I0pmoScq0UI90_UjQ=w1333-h1000-k-no

https://github.com/PacktPublishing/Advanced-Deep-Learning-with-Keras

https://github.com/xprilion/Submitty

https://summerofcode.withgoogle.com/programs/2024/projects

In [6]:
bali_flight = {
   "search_metadata":{
      "id":"66eaac76123457b97c787573",
      "status":"Success",
      "json_endpoint":"https://serpapi.com/searches/a308ad8b26a7171f/66eaac76123457b97c787573.json",
      "created_at":"2024-09-18 10:33:26 UTC",
      "processed_at":"2024-09-18 10:33:26 UTC",
      "google_flights_url":"https://www.google.com/travel/flights?hl=en&gl=us&curr=USD&tfs=CBwQAhoeEgoyMDI0LTA5LTE4agcIARIDS0hJcgcIARIDRFBTGh4SCjIwMjQtMDktMjVqBwgBEgNEUFNyBwgBEgNLSElCBQEBAgIESAFwAZgBAQ==",
      "raw_html_file":"https://serpapi.com/searches/a308ad8b26a7171f/66eaac76123457b97c787573.html",
      "prettify_html_file":"https://serpapi.com/searches/a308ad8b26a7171f/66eaac76123457b97c787573.prettify",
      "total_time_taken":1.81
   },
   "search_parameters":{
      "engine":"google_flights",
      "hl":"en",
      "gl":"us",
      "type":"1",
      "departure_id":"KHI",
      "arrival_id":"DPS",
      "outbound_date":"2024-09-18",
      "return_date":"2024-09-25",
      "travel_class":1,
      "adults":2,
      "children":2,
      "infants_in_seat":1,
      "stops":0,
      "currency":"USD"
   },
   "other_flights":[
      {
         "flights":[
            {
               "departure_airport":{
                  "name":"Jinnah International Airport",
                  "id":"KHI",
                  "time":"2024-09-18 22:55"
               },
               "arrival_airport":{
                  "name":"Dubai International Airport",
                  "id":"DXB",
                  "time":"2024-09-19 00:10"
               },
               "duration":135,
               "airplane":"Boeing 777",
               "airline":"Emirates",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/EK.png",
               "travel_class":"Economy",
               "flight_number":"EK 603",
               "legroom":"32 in",
               "extensions":[
                  "Above average legroom (32 in)",
                  "Wi-Fi for a fee",
                  "In-seat power & USB outlets",
                  "On-demand video",
                  "Carbon emissions estimate: 689 kg"
               ]
            },
            {
               "departure_airport":{
                  "name":"Dubai International Airport",
                  "id":"DXB",
                  "time":"2024-09-19 10:25"
               },
               "arrival_airport":{
                  "name":"Kuala Lumpur International Airport",
                  "id":"KUL",
                  "time":"2024-09-19 21:50"
               },
               "duration":445,
               "airplane":"Airbus A380",
               "airline":"Emirates",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/EK.png",
               "travel_class":"Economy",
               "flight_number":"EK 342",
               "legroom":"32 in",
               "extensions":[
                  "Above average legroom (32 in)",
                  "Wi-Fi for a fee",
                  "In-seat power & USB outlets",
                  "On-demand video",
                  "Carbon emissions estimate: 2359 kg"
               ],
               "often_delayed_by_over_30_min":True
            },
            {
               "departure_airport":{
                  "name":"Kuala Lumpur International Airport",
                  "id":"KUL",
                  "time":"2024-09-20 14:00"
               },
               "arrival_airport":{
                  "name":"I Gusti Ngurah Rai International Airport",
                  "id":"DPS",
                  "time":"2024-09-20 17:00"
               },
               "duration":180,
               "airplane":"Boeing 737",
               "airline":"Batik Air",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/OD.png",
               "travel_class":"Economy",
               "flight_number":"OD 193",
               "ticket_also_sold_by":[
                  "Emirates"
               ],
               "legroom":"32 in",
               "extensions":[
                  "Above average legroom (32 in)",
                  "In-seat USB outlet",
                  "On-demand video",
                  "Carbon emissions estimate: 910 kg"
               ]
            }
         ],
         "layovers":[
            {
               "duration":615,
               "name":"Dubai International Airport",
               "id":"DXB",
               "overnight":True
            },
            {
               "duration":970,
               "name":"Kuala Lumpur International Airport",
               "id":"KUL",
               "overnight":True
            }
         ],
         "total_duration":2345,
         "carbon_emissions":{
            "this_flight":3959000,
            "typical_for_this_route":3675000,
            "difference_percent":8
         },
         "price":3718,
         "type":"Round trip",
         "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/multi.png",
         "departure_token":"WyJDalJJTFROeldGQkxaV2RIV21OQlJUVklMV2RDUnkwdExTMHRMUzB0TFc5NVkycHJOMEZCUVVGQlIySnhja2hqUTBKbmQyVkJFaEZGU3pZd00zeEZTek0wTW54UFJERTVNeG9MQ00zWUZoQUNHZ05WVTBRNEhIRE4yQlk9IixbWyJLSEkiLCIyMDI0LTA5LTE4IiwiRFhCIixudWxsLCJFSyIsIjYwMyJdLFsiRFhCIiwiMjAyNC0wOS0xOSIsIktVTCIsbnVsbCwiRUsiLCIzNDIiXSxbIktVTCIsIjIwMjQtMDktMjAiLCJEUFMiLG51bGwsIk9EIiwiMTkzIl1dXQ=="
      },
      {
         "flights":[
            {
               "departure_airport":{
                  "name":"Jinnah International Airport",
                  "id":"KHI",
                  "time":"2024-09-18 22:55"
               },
               "arrival_airport":{
                  "name":"Dubai International Airport",
                  "id":"DXB",
                  "time":"2024-09-19 00:10"
               },
               "duration":135,
               "airplane":"Boeing 777",
               "airline":"Emirates",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/EK.png",
               "travel_class":"Economy",
               "flight_number":"EK 603",
               "legroom":"32 in",
               "extensions":[
                  "Above average legroom (32 in)",
                  "Wi-Fi for a fee",
                  "In-seat power & USB outlets",
                  "On-demand video",
                  "Carbon emissions estimate: 689 kg"
               ]
            },
            {
               "departure_airport":{
                  "name":"Dubai International Airport",
                  "id":"DXB",
                  "time":"2024-09-19 03:25"
               },
               "arrival_airport":{
                  "name":"I Gusti Ngurah Rai International Airport",
                  "id":"DPS",
                  "time":"2024-09-19 16:35"
               },
               "duration":550,
               "airplane":"Airbus A380",
               "airline":"Emirates",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/EK.png",
               "travel_class":"Economy",
               "flight_number":"EK 368",
               "legroom":"32 in",
               "extensions":[
                  "Above average legroom (32 in)",
                  "Wi-Fi for a fee",
                  "In-seat power & USB outlets",
                  "On-demand video",
                  "Carbon emissions estimate: 3192 kg"
               ],
               "overnight":True
            }
         ],
         "layovers":[
            {
               "duration":195,
               "name":"Dubai International Airport",
               "id":"DXB",
               "overnight":True
            }
         ],
         "total_duration":880,
         "carbon_emissions":{
            "this_flight":3882000,
            "typical_for_this_route":3675000,
            "difference_percent":6
         },
         "price":3884,
         "type":"Round trip",
         "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/EK.png",
         "departure_token":"WyJDalJJTFROeldGQkxaV2RIV21OQlJUVklMV2RDUnkwdExTMHRMUzB0TFc5NVkycHJOMEZCUVVGQlIySnhja2hqUTBKbmQyVkJFZ3RGU3pZd00zeEZTek0yT0JvTENLM2FGeEFDR2dOVlUwUTRISEN0MmhjPSIsW1siS0hJIiwiMjAyNC0wOS0xOCIsIkRYQiIsbnVsbCwiRUsiLCI2MDMiXSxbIkRYQiIsIjIwMjQtMDktMTkiLCJEUFMiLG51bGwsIkVLIiwiMzY4Il1dXQ=="
      },
      {
         "flights":[
            {
               "departure_airport":{
                  "name":"Jinnah International Airport",
                  "id":"KHI",
                  "time":"2024-09-18 23:55"
               },
               "arrival_airport":{
                  "name":"Dubai International Airport",
                  "id":"DXB",
                  "time":"2024-09-19 01:10"
               },
               "duration":135,
               "airplane":"Boeing 737",
               "airline":"flydubai",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/FZ.png",
               "travel_class":"Economy",
               "flight_number":"FZ 330",
               "ticket_also_sold_by":[
                  "Emirates"
               ],
               "legroom":"29 in",
               "extensions":[
                  "Below average legroom (29 in)",
                  "Stream media to your device",
                  "Carbon emissions estimate: 548 kg"
               ]
            },
            {
               "departure_airport":{
                  "name":"Dubai International Airport",
                  "id":"DXB",
                  "time":"2024-09-19 03:25"
               },
               "arrival_airport":{
                  "name":"I Gusti Ngurah Rai International Airport",
                  "id":"DPS",
                  "time":"2024-09-19 16:35"
               },
               "duration":550,
               "airplane":"Airbus A380",
               "airline":"Emirates",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/EK.png",
               "travel_class":"Economy",
               "flight_number":"EK 368",
               "legroom":"32 in",
               "extensions":[
                  "Above average legroom (32 in)",
                  "Wi-Fi for a fee",
                  "In-seat power & USB outlets",
                  "On-demand video",
                  "Carbon emissions estimate: 3192 kg"
               ],
               "overnight":True
            }
         ],
         "layovers":[
            {
               "duration":135,
               "name":"Dubai International Airport",
               "id":"DXB",
               "overnight":True
            }
         ],
         "total_duration":820,
         "carbon_emissions":{
            "this_flight":3741000,
            "typical_for_this_route":3675000,
            "difference_percent":2
         },
         "price":3884,
         "type":"Round trip",
         "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/multi.png",
         "departure_token":"WyJDalJJTFROeldGQkxaV2RIV21OQlJUVklMV2RDUnkwdExTMHRMUzB0TFc5NVkycHJOMEZCUVVGQlIySnhja2hqUTBKbmQyVkJFZ3RHV2pNek1IeEZTek0yT0JvTENLM2FGeEFDR2dOVlUwUTRISEN0MmhjPSIsW1siS0hJIiwiMjAyNC0wOS0xOCIsIkRYQiIsbnVsbCwiRloiLCIzMzAiXSxbIkRYQiIsIjIwMjQtMDktMTkiLCJEUFMiLG51bGwsIkVLIiwiMzY4Il1dXQ=="
      },
      {
         "flights":[
            {
               "departure_airport":{
                  "name":"Jinnah International Airport",
                  "id":"KHI",
                  "time":"2024-09-18 23:55"
               },
               "arrival_airport":{
                  "name":"Dubai International Airport",
                  "id":"DXB",
                  "time":"2024-09-19 01:10"
               },
               "duration":135,
               "airplane":"Boeing 737",
               "airline":"flydubai",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/FZ.png",
               "travel_class":"Economy",
               "flight_number":"FZ 330",
               "ticket_also_sold_by":[
                  "Emirates"
               ],
               "legroom":"29 in",
               "extensions":[
                  "Below average legroom (29 in)",
                  "Stream media to your device",
                  "Carbon emissions estimate: 548 kg"
               ]
            },
            {
               "departure_airport":{
                  "name":"Dubai International Airport",
                  "id":"DXB",
                  "time":"2024-09-19 10:25"
               },
               "arrival_airport":{
                  "name":"Kuala Lumpur International Airport",
                  "id":"KUL",
                  "time":"2024-09-19 21:50"
               },
               "duration":445,
               "airplane":"Airbus A380",
               "airline":"Emirates",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/EK.png",
               "travel_class":"Economy",
               "flight_number":"EK 342",
               "legroom":"32 in",
               "extensions":[
                  "Above average legroom (32 in)",
                  "Wi-Fi for a fee",
                  "In-seat power & USB outlets",
                  "On-demand video",
                  "Carbon emissions estimate: 2359 kg"
               ],
               "often_delayed_by_over_30_min":True
            },
            {
               "departure_airport":{
                  "name":"Kuala Lumpur International Airport",
                  "id":"KUL",
                  "time":"2024-09-20 09:00"
               },
               "arrival_airport":{
                  "name":"I Gusti Ngurah Rai International Airport",
                  "id":"DPS",
                  "time":"2024-09-20 12:05"
               },
               "duration":185,
               "airplane":"Airbus A330",
               "airline":"Malaysia Airlines",
               "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/MH.png",
               "travel_class":"Economy",
               "flight_number":"MH 715",
               "legroom":"32 in",
               "extensions":[
                  "Above average legroom (32 in)",
                  "In-seat power & USB outlets",
                  "On-demand video",
                  "Carbon emissions estimate: 874 kg"
               ],
               "often_delayed_by_over_30_min":True
            }
         ],
         "layovers":[
            {
               "duration":555,
               "name":"Dubai International Airport",
               "id":"DXB",
               "overnight":True
            },
            {
               "duration":670,
               "name":"Kuala Lumpur International Airport",
               "id":"KUL",
               "overnight":True
            }
         ],
         "total_duration":1990,
         "carbon_emissions":{
            "this_flight":3783000,
            "typical_for_this_route":3675000,
            "difference_percent":3
         },
         "price":4269,
         "type":"Round trip",
         "airline_logo":"https://www.gstatic.com/flights/airline_logos/70px/multi.png",
         "departure_token":"WyJDalJJTFROeldGQkxaV2RIV21OQlJUVklMV2RDUnkwdExTMHRMUzB0TFc5NVkycHJOMEZCUVVGQlIySnhja2hqUTBKbmQyVkJFaEZHV2pNek1IeEZTek0wTW54TlNEY3hOUm9MQ0xTR0doQUNHZ05WVTBRNEhIQzBoaG89IixbWyJLSEkiLCIyMDI0LTA5LTE4IiwiRFhCIixudWxsLCJGWiIsIjMzMCJdLFsiRFhCIiwiMjAyNC0wOS0xOSIsIktVTCIsbnVsbCwiRUsiLCIzNDIiXSxbIktVTCIsIjIwMjQtMDktMjAiLCJEUFMiLG51bGwsIk1IIiwiNzE1Il1dXQ=="
      }
   ],
   "airports":[
      {
         "departure":[
            {
               "airport":{
                  "id":"KHI",
                  "name":"Jinnah International Airport"
               },
               "city":"Karachi",
               "country":"Pakistan",
               "country_code":"PK",
               "image":"https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSTRWvWM08EzPR06ONmd9A6jGIHUCpoDaOMyrxQqkSTF9X8V1haOjy4Z1gUcpOxgR9fuc1g6fFvfbPBew",
               "thumbnail":"https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSVZgSeD37LtJ7PsxIAQ4d4QUbVzkjF7PF5oaE8KuGq5DEJhWbIMzWf-0Nk0aXtPyxR85tmWtD9vbXRniM6ti0ncBr8mWk7kRPpwrH6OGw"
            }
         ],
         "arrival":[
            {
               "airport":{
                  "id":"DPS",
                  "name":"I Gusti Ngurah Rai International Airport"
               },
               "city":"Denpasar",
               "country":"Indonesia",
               "country_code":"ID",
               "image":"https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcSpjCo287YKiDxrbQlSm3Af70o71dUjCPZ6_dVDyf-1TqPjvjrZ37t33Q_7iDHhYS48wNu0LJdENRBRSg",
               "thumbnail":"https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcQFsxUxjHvh4B-OC3MBZsppnugcvv5-mwb1QyUnDEw4ERAN8pQGYpPooKz04neOm3ivuxDBxp0nyrI4S_UEFevemYjK9tdUmJgtUDr1HgQ"
            }
         ]
      },
      {
         "departure":[
            {
               "airport":{
                  "id":"DPS",
                  "name":"I Gusti Ngurah Rai International Airport"
               },
               "city":"Denpasar",
               "country":"Indonesia",
               "country_code":"ID",
               "image":"https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcSpjCo287YKiDxrbQlSm3Af70o71dUjCPZ6_dVDyf-1TqPjvjrZ37t33Q_7iDHhYS48wNu0LJdENRBRSg",
               "thumbnail":"https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcQFsxUxjHvh4B-OC3MBZsppnugcvv5-mwb1QyUnDEw4ERAN8pQGYpPooKz04neOm3ivuxDBxp0nyrI4S_UEFevemYjK9tdUmJgtUDr1HgQ"
            }
         ],
         "arrival":[
            {
               "airport":{
                  "id":"KHI",
                  "name":"Jinnah International Airport"
               },
               "city":"Karachi",
               "country":"Pakistan",
               "country_code":"PK",
               "image":"https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSTRWvWM08EzPR06ONmd9A6jGIHUCpoDaOMyrxQqkSTF9X8V1haOjy4Z1gUcpOxgR9fuc1g6fFvfbPBew",
               "thumbnail":"https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSVZgSeD37LtJ7PsxIAQ4d4QUbVzkjF7PF5oaE8KuGq5DEJhWbIMzWf-0Nk0aXtPyxR85tmWtD9vbXRniM6ti0ncBr8mWk7kRPpwrH6OGw"
            }
         ]
      }
   ]
}

In [7]:
bali_other_flights = bali_flight["other_flights"]

In [11]:
Markdown(parse_flight_data(bali_other_flights[0]))


                      Flight Number: EK 603
                      Airline: Emirates
                      Airplane: Boeing 777
                      Class: Economy
                      Legroom: 32 in
                      Duration: 135 minutes
                      Departure: Jinnah International Airport (KHI) at 2024-09-18 22:55
                      Arrival: Dubai International Airport (DXB) at 2024-09-19 00:10
                      Extensions: Above average legroom (32 in), Wi-Fi for a fee, In-seat power & USB outlets, On-demand video, Carbon emissions estimate: 689 kg
                      

                      Flight Number: EK 342
                      Airline: Emirates
                      Airplane: Airbus A380
                      Class: Economy
                      Legroom: 32 in
                      Duration: 445 minutes
                      Departure: Dubai International Airport (DXB) at 2024-09-19 10:25
                      Arrival: Kuala Lumpur International Airport (KUL) at 2024-09-19 21:50
                      Extensions: Above average legroom (32 in), Wi-Fi for a fee, In-seat power & USB outlets, On-demand video, Carbon emissions estimate: 2359 kg
                      

                      Flight Number: OD 193
                      Airline: Batik Air
                      Airplane: Boeing 737
                      Class: Economy
                      Legroom: 32 in
                      Duration: 180 minutes
                      Departure: Kuala Lumpur International Airport (KUL) at 2024-09-20 14:00
                      Arrival: I Gusti Ngurah Rai International Airport (DPS) at 2024-09-20 17:00
                      Extensions: Above average legroom (32 in), In-seat USB outlet, On-demand video, Carbon emissions estimate: 910 kg
                      

        Total Trip Duration: 2345 minutes
        Carbon Emissions: 3959000 kg (Typical for this route: 3675000 kg, Difference: 8%)
        Total Price: $3718
        Trip Type: Round trip
        

In [12]:
Markdown(parse_flight_data(bali_other_flights[1]))


                      Flight Number: EK 603
                      Airline: Emirates
                      Airplane: Boeing 777
                      Class: Economy
                      Legroom: 32 in
                      Duration: 135 minutes
                      Departure: Jinnah International Airport (KHI) at 2024-09-18 22:55
                      Arrival: Dubai International Airport (DXB) at 2024-09-19 00:10
                      Extensions: Above average legroom (32 in), Wi-Fi for a fee, In-seat power & USB outlets, On-demand video, Carbon emissions estimate: 689 kg
                      

                      Flight Number: EK 368
                      Airline: Emirates
                      Airplane: Airbus A380
                      Class: Economy
                      Legroom: 32 in
                      Duration: 550 minutes
                      Departure: Dubai International Airport (DXB) at 2024-09-19 03:25
                      Arrival: I Gusti Ngurah Rai International Airport (DPS) at 2024-09-19 16:35
                      Extensions: Above average legroom (32 in), Wi-Fi for a fee, In-seat power & USB outlets, On-demand video, Carbon emissions estimate: 3192 kg
                      This flight is overnight.


        Total Trip Duration: 880 minutes
        Carbon Emissions: 3882000 kg (Typical for this route: 3675000 kg, Difference: 6%)
        Total Price: $3884
        Trip Type: Round trip
        